In [ ]:
# !pip install info-nce-pytorch

In [ ]:
!wget https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip

In [ ]:
!nvidia-smi

In [ ]:
!wget https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip

In [ ]:
!ls

In [ ]:
!unzip -q ./Flickr8k_Dataset.zip

In [ ]:
!unzip -q ./Flickr8k_text.zip

In [ ]:
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
import random
import wandb
import torch
import numpy as np
import os
torch.use_deterministic_algorithms(True)
def set_seed(seed):

    random.seed(seed)     # python random generator
    np.random.seed(seed)  # numpy random generator

    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:

from PIL import Image


In [ ]:
#raw_image = Image.open('./Flicker8k_Dataset/1001773457_577c3a7d70.jpg')

In [ ]:
#raw_image

In [ ]:
!pip install git+https://github.com/openai/CLIP.git
    
    

In [ ]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# image = preprocess(Image.open('./Flicker8k_Dataset/1001773457_577c3a7d70.jpg')).unsqueeze(0).to(device)
# text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)

# with torch.no_grad():
#     image_features = model.encode_image(image)
#     text_features = model.encode_text(text)

#     logits_per_image, logits_per_text = model(image, text)
#     probs = logits_per_image.softmax(dim=-1).cpu().numpy()

# print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

In [ ]:
# model

In [ ]:
image_features.shape

In [ ]:
text_features.shape

In [ ]:
# import torch
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# from lavis.models import load_model_and_preprocess
# model, vis_processors, txt_processors = load_model_and_preprocess(name="clip_feature_extractor", model_type="base", is_eval=True, device=device)

In [ ]:
# def get_tensor(caption, raw_image):
#     image = preprocess(raw_image).unsqueeze(0).to(device)
#     text = clip.tokenize([caption]).to(device)

#     with torch.no_grad():
#         image_features = model.encode_image(image)
#         text_features = model.encode_text(text)

#     return image_features.squeeze(0).cpu(), text_features.squeeze(0).cpu()

In [ ]:
# with open('./Flickr8k.token.txt' , 'r') as f:
#     kk = f.readlines()

In [ ]:
from tqdm import tqdm

In [ ]:
img_tensor = []
txt_tensor = []
set_seed(42)
#for i in tqdm(range(len(kk))):
for i in tqdm(range(800)):
    nos = kk[i].split('\t')[0].split('#')[1]
    # print(nos)
    if nos=='1':
        img = kk[i].split('\t')[0].split('#')[0]
        txt = kk[i].split('\t')[1].strip().lower()
        try:
            raw_image = Image.open('./Flicker8k_Dataset/'+img)
        except:
            print(1)
            continue
        it, tt = get_tensor(txt, raw_image)
        img_tensor.append(it)
        txt_tensor.append(tt)


In [ ]:
img_tensor = torch.stack(img_tensor)

In [ ]:
img_tensor.shape


In [ ]:
txt_tensor = torch.stack(txt_tensor)

In [ ]:

txt_tensor.shape

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, tensor1, tensor2):
        self.tensor1 = tensor1
        self.tensor2 = tensor2

    def __len__(self):
        return len(self.tensor1)

    def __getitem__(self, idx):
        return self.tensor1[idx], self.tensor2[idx]

def collate_fn(batch):
    tensor1_batch, tensor2_batch = zip(*batch)
    return torch.stack(tensor1_batch), torch.stack(tensor2_batch)

# Create your tensors
N = 100  # Example number of samples
tensor1 = img_tensor
tensor2 = txt_tensor

# Create a custom dataset
custom_dataset = CustomDataset(tensor1, tensor2)

# Create a DataLoader with your collate_fn
batch_size = 32
dataloader = DataLoader(custom_dataset, batch_size=batch_size, collate_fn=collate_fn)

# # Iterate through the DataLoader
# for batch_tensor1, batch_tensor2 in dataloader:
#     print("Tensor 1 batch shape:", batch_tensor1.shape)
#     print("Tensor 2 batch shape:", batch_tensor2.shape)
#     print("-" * 30)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

# Simulated batch sizes and vector dimensions
batch_size = 32
vector_dim = 1024

In [ ]:
# mdl = nn.Sequential(nn.Linear(vector_dim, 256), nn.ReLU(), nn.Linear(256, 512)).to(device)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class fusion(nn.Module):
    def __init__(self,img_feat_size, txt_feat_size, is_first, K, O, DROPOUT_R):
        super(fusion, self).__init__()
        #self.__C = __C
        self.K = K
        self.O = O
        self.DROPOUT_R = DROPOUT_R

        self.is_first = is_first
        self.proj_i = nn.Linear(img_feat_size, K * O)
        self.proj_t = nn.Linear(txt_feat_size, K * O)

        self.dropout = nn.Dropout(DROPOUT_R)
        self.pool = nn.AvgPool1d(K, stride = K)

    def forward(self, img_feat, txt_feat, exp_in=1):

        batch_size = img_feat.shape[0]
        img_feat = self.proj_i(img_feat)
        txt_feat = self.proj_t(txt_feat)

        exp_out = img_feat * txt_feat
        exp_out = self.dropout(exp_out) if self.is_first else self.dropout(exp_out * exp_in)
        z = self.pool(exp_out) * self.K
        z = F.normalize(z.view(batch_size, -1))
        z = z.view(batch_size, -1, self.O)
        return z

In [ ]:
# optimizer = optim.Adam(mdl.parameters(), lr=1e-5)


# # Training loop
# num_epochs = 500
# def transpose(x):
#     return x.transpose(-2, -1)

# for epoch in tqdm(range(num_epochs)):
#     l=0
#     cnt = 0
#     for image_tensor, text_tensor in dataloader:
#         # print(batch_model1, batch_model1)
#         image_tensor = image_tensor.unsqueeze(1).float().to(device)
#         text_tensor = text_tensor.unsqueeze(1).float().to(device)
#         # joint_tensor = torch.cat((image_tensor, text_tensor), dim=1)
#         # print(image_tensor)
#         multimodal_tensor = mdl(image_tensor, text_tensor).squeeze(dim=1)

#         #Normalizing both of the vectors to unit dimension
#         p = torch.nn.functional.normalize(multimodal_tensor, dim=-1)
#         q = torch.nn.functional.normalize(image_tensor.squeeze(dim=1), dim=-1)
#         r = torch.nn.functional.normalize(text_tensor.squeeze(dim=1), dim=-1)


#         labels = torch.arange(len(p)).to(device)


#         logits = p @ transpose(q)
#         temperature = 0.8
#         loss_0 = torch.nn.functional.cross_entropy(logits / temperature, labels, reduction='mean')


#         logits_1 = p @ transpose(r)
#         temperature = 0.8
#         loss_1 = torch.nn.functional.cross_entropy(logits_1 / temperature, labels, reduction='mean')


#         loss = loss_0+loss_1


#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()

#         l+=loss.detach().item()
#         cnt+=1

#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {l/cnt}")

# print("Training complete!")


In [ ]:
!ls


In [ ]:
import os

# {"username":"newcodevelop","key":"4ee1aba876485ad4ff02791b2959a664"}
# os.environ["KAGGLE_CONFIG_DIR"] = "/content/"
os.environ["KAGGLE_USERNAME"]="newcodevelop"
os.environ["KAGGLE_KEY"]="bcc404580ff37b2f37118ee42a7cc1fe4"

In [ ]:
!kaggle datasets download -d parthplc/facebook-hateful-meme-dataset

In [ ]:
!ls

In [ ]:
!unzip -q ./facebook-hateful-meme-dataset.zip

In [ ]:
# %cd /kaggle/working/data

In [ ]:
# !grep -r "dick"

In [ ]:
from PIL import Image
im = Image.open("/kaggle/input/facebook-hateful-meme-dataset/data/img/05471.png")

In [ ]:
im.save('picc.jpg')

In [ ]:
!pip install jsonlines

In [ ]:
import jsonlines

In [ ]:
# im_tensor,tx_tensor = [],[]
# gold_label = []
# img_id = []
# counter = 0
# set_seed(42)
# with jsonlines.open('/kaggle/input/facebook-hateful-meme-dataset/data/train.jsonl') as f:
#       for line in tqdm(f):
#         # print(line)

#         hypo = line['text']

#         raw_image = Image.open('./data/'+str(line['img']))
#         try:
#             it, tt = get_tensor(hypo, raw_image)
#         except:
#             continue
#         im_tensor.append(it)
#         tx_tensor.append(tt)
#         gold_label.append(int(line['label']))
#         img_id.append(str(line['img']))
#         counter+=1
#         if counter==8500:
#             break


In [ ]:
# im_tensor_,tx_tensor_ = [],[]
# img_id_ = []
# gold_label_ = []
# counter = 0
# set_seed(42)
# with jsonlines.open('/kaggle/input/facebook-hateful-meme-dataset/data/dev.jsonl') as f:
#       for line in tqdm(f):

#         hypo = line['text']
#         gold_label_.append(int(line['label']))
#         raw_image = Image.open('./data/'+str(line['img']))
#         img_id_.append(str(line['img']))
#         it, tt = get_tensor(hypo, raw_image)
#         im_tensor_.append(it)
#         tx_tensor_.append(tt)



In [ ]:
from tqdm import tqdm

In [ ]:
id2text = {}
with jsonlines.open('/kaggle/input/facebook-hateful-meme-dataset/data/dev.jsonl') as f:
    for line in tqdm(f):

        id2text[str(line['img']).split('/')[1]] = line['text']
    
        


In [ ]:
from tqdm import tqdm

In [ ]:
id2text = {}
with jsonlines.open('/kaggle/input/facebook-hateful-meme-dataset/data/train.jsonl') as f:
    for line in tqdm(f):

        id2text[str(line['img']).split('/')[1]] = int(line['label'])
    
        


In [ ]:
id2text

In [ ]:
len(id2text)

In [ ]:
#!pip install gdown

In [ ]:
#import gdown

In [ ]:
# url = 'https://drive.google.com/u/0/uc?id=1JPQA1jcmfDYkEK_89U_UP-VtyUrTwJKi'
# output = 'entity_json.zip'

In [ ]:
# gdown.download(url, output, quiet=False)

In [ ]:
# !unzip -q ./entity_json.zip

In [ ]:
import os
import json
import spacy
import pickle
from tqdm import tqdm
import requests
import pandas as pd

#nlp = spacy.load("en_core_web_sm")

In [ ]:

# with open('./caption_fb_dataset.pickle', 'rb') as h:
#      caption = pickle.load(h)

# print(caption)

# loef = os.listdir('./entity_json')
# ll = []
# for i in loef:
#     candidate = i.split('.')
#     if len(candidate)==3:
#         ll.append(candidate[0]+'.0.json')
#     else:
#         ll.append(i)

# loef = list(set(ll))

# print(len(ll))
# print(len(loef))

In [ ]:



# # exit(0)
# TAGS = {}
# cnt=0
# bad_names = []
# for i_ in tqdm(loef):
#     f = open(os.path.join(os.getcwd(),'entity_json/'+i_))
#     k = json.load(f)
#     # print('*****************************')
#     # print(caption[i_.split('.')[0]+'.png'])

#     # print(k['bestGuessLabels'])

#     try:

#         ners = []
#         for i in range(len(k['webEntities'])):

#             try:
#                 doc = nlp(k['webEntities'][i]['description'])
#                 for ent in doc.ents:
#                     # print(ent.text, ent.start_char, ent.end_char, ent.label_)
#                     ners.append((ent.text, ent.label_))

#             except:
#                 break
#         if len(i_.split('.'))==3:
#             TAGS[i_.split('.')[0]+'.'+i_.split('.')[2]] = ners
#         else:
#             TAGS[i_.split('.')[0]+'.'+i_.split('.')[1]] = ners

#         # print(TAGS)
#     except:
#         print('in except')
#         if len(i_.split('.'))==3:
#             bad_names.append(i_.split('.')[0]+'.'+i_.split('.')[2])
#         else:
#             bad_names.append(i_.split('.')[0]+'.'+i_.split('.')[1])


#     f.close()
#     # cnt+=1
#     # if cnt==15:
#     #     break

# # print(TAGS)
# print('total in except {}'.format(len(bad_names)))
# print(bad_names)

# PROMPTS = {}
# for i_ in tqdm(caption):

#     fname = i_.split('.')[0]+'.json'
#     # print(fname)
#     if fname in bad_names:
#         continue


#     tags = [i[0] for i in TAGS[fname]]
#     tags_ = 'I have entities like'
#     for i in tags:
#         tags_+= ' '+i+','

#     tags = tags_
#     # print(tags)
#     PROMPTS[i_] = tags

In [ ]:
import torch

In [ ]:
kb_fb = torch.load('/kaggle/input/kb-fb-ds/kb_fb.pt')

In [ ]:
# kb_fb

In [ ]:
device1 = 'cuda:1'

In [ ]:
im_tensor = torch.stack(im_tensor)
tx_tensor = torch.stack(tx_tensor)
gl = gold_label

In [ ]:

im_tensor_ = torch.stack(im_tensor_)
tx_tensor_ = torch.stack(tx_tensor_)
gl_ = gold_label_

In [ ]:
# Save to file

# torch.save(im_tensor, 'im_tensor.pt')
# torch.save(im_tensor_, 'im_tensor_.pt')
# torch.save(tx_tensor, 'tx_tensor.pt')
# torch.save(tx_tensor_, 'tx_tensor_.pt')

# torch.save(gl, 'gl.pt')
# torch.save(gl_, 'gl_.pt')

# torch.save(img_id, 'img_id.pt')
# torch.save(img_id_, 'img_id_.pt')

In [ ]:
prefix = '/kaggle/input/tensors/'
im_tensor = torch.load(prefix+'im_tensor.pt')
im_tensor_ = torch.load(prefix+'im_tensor_.pt')
tx_tensor = torch.load(prefix+'tx_tensor.pt')
tx_tensor_ = torch.load(prefix+'tx_tensor_.pt')

gl = torch.load(prefix+'gl.pt')
gl_ = torch.load(prefix+'gl_.pt')

img_id = torch.load(prefix+'img_id.pt')
img_id_ = torch.load(prefix+'img_id_.pt')

In [ ]:
im_tensor[0:2,:]

In [ ]:
im_tensor_[0:2,:]

In [ ]:
im_tensor.shape, tx_tensor.shape


In [ ]:
len(gl)

In [ ]:
import torch

class TinyModel(torch.nn.Module):

    def __init__(self, mdl, mdl_rand, rand=False):
        super(TinyModel, self).__init__()
        if rand:
            self.linear1 = nn.Sequential(nn.Linear(1024, 256), nn.ReLU(), nn.Linear(256, 512))
        else:
            self.linear1 = mdl_rand
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(512, 128)
        self.linear3 = torch.nn.Linear(128, 2)
        self.rand = rand
        self.softmax = torch.nn.Softmax()
        self.proj = torch.nn.Linear(512,768)
        # self.proj = torch.nn.Linear(512,5120)

    def forward(self, x, y):
        joint_tensor = torch.cat((x, y), dim=1)
        if self.rand:
            m_ = self.linear1(joint_tensor)
        else:
            x = x.unsqueeze(1)
            y = y.unsqueeze(1)
            m_ = self.linear1(x, y).squeeze(dim=1)
        m = self.activation(m_)
        m = self.linear3(self.linear2(m))
        # m = self.softmax(m)
        return m, self.proj(m_)



In [ ]:
print(im_tensor.shape, tx_tensor.shape, len(gl), len(img_id))

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, tensor1, tensor2, gold_label, ids):
        self.tensor1 = tensor1
        self.tensor2 = tensor2
        self.gl = gold_label
        self.ids = ids

    def __len__(self):
        return len(self.tensor1)

    def __getitem__(self, idx):
        return self.tensor1[idx], self.tensor2[idx], self.gl[idx], self.ids[idx]

# def collate_fn(batch):
#     tensor1_batch, tensor2_batch = zip(*batch)
#     return torch.stack(tensor1_batch), torch.stack(tensor2_batch)

# Create your tensors
N = 100  # Example number of samples
tensor1 = im_tensor
tensor2 = tx_tensor

# Create a custom dataset
custom_dataset = CustomDataset(tensor1, tensor2, gl, img_id)
train_size = int(0.8 * len(custom_dataset))
test_size = len(custom_dataset) - train_size
# torch.manual_seed(42)
set_seed(42)
train_dataset, test_dataset = torch.utils.data.random_split(custom_dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))

# Create a DataLoader with your collate_fn
batch_size = 4
import torch
torch.manual_seed(42)


def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(42)

set_seed(42)
dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    worker_init_fn=seed_worker,
    generator=g,
)



# dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)

# Iterate through the DataLoader
counter = 0
for batch_tensor1, batch_tensor2, b3, b4 in dataloader:
    print("Tensor 1 batch shape:", batch_tensor1.shape)
    print("Tensor 2 batch shape:", batch_tensor2.shape)
    print("Tensor 3 batch shape:", b3.shape)
    print("Tensor 4 batch shape:", b4)
    print("-" * 30)
    counter+=1
    if counter==10:
        break


In [ ]:
!pip install transformers

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
# !pip install auto-gptq

In [ ]:
model1_name = 'gpt2'
tokenizer1 = GPT2Tokenizer.from_pretrained(model1_name)
model1 = GPT2LMHeadModel.from_pretrained(model1_name,output_hidden_states=True).to(device)
mdl_rand = fusion(512,512,True,256,512,0.1).to(device)
tinymodel = TinyModel(mdl,mdl_rand,rand=False).to(device)

In [ ]:
!nvidia-smi

In [ ]:
# from transformers import GPT2Tokenizer, GPT2LMHeadModel, AutoTokenizer
# from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
# quantized_model_dir = "TheBloke/stable-vicuna-13B-GPTQ"

# model_basename = "stable-vicuna-13B-GPTQ-4bit.compat.no-act-order"

# use_triton = False

# tokenizer1 = AutoTokenizer.from_pretrained(quantized_model_dir, use_fast=True, max_lengh=128)

# quantize_config = BaseQuantizeConfig(
# bits=4,
# group_size=128,
# desc_act=False
# )

# model1 = AutoGPTQForCausalLM.from_quantized(quantized_model_dir,
# use_safetensors=True,
# model_basename=model_basename,
# device=device1,
# use_triton=use_triton,
# quantize_config=quantize_config)

In [ ]:
# def get_tokens(prpmt,begin=False):
#   if begin:
#     prepended_inp = [tokenizer1.encode(i) for i in prpmt]
#   else:
#     prepended_inp = [tokenizer1.encode(i)[1:] for i in prpmt]
#   fin = []
#   for i in prepended_inp:
#     inter = []
#     for j in i:
#       inter.append(E[j,:])
#     fin.append(torch.stack(inter))
#   return torch.stack(fin), torch.tensor(prepended_inp)


In [ ]:
def get_tokens(prpmt,begin=False):
    set_seed(42)
    if begin:
        prepended_inp = [tokenizer1.encode(i) for i in prpmt]
    else:
        prepended_inp = [tokenizer1.encode(i) for i in prpmt]
    max_len = max([len(i) for i in prepended_inp])
    #print(max_len)
    attn_mask = []
    bs = len(prpmt)
    for i in range(bs):
        tmp_len = max_len - len(prepended_inp[i])
        tmp_mask = torch.tensor([1]* len(prepended_inp[i]) + [0]* tmp_len)
        attn_mask.append(tmp_mask)
        extra_tokens = tokenizer1.encode(tokenizer1.eos_token)*tmp_len
        prepended_inp[i] = prepended_inp[i]+extra_tokens
    
    attn_mask = torch.stack(attn_mask)
    #print(prepended_inp)
    #print(attn_mask, attn_mask.shape)
    fin = []
    for i in prepended_inp:
        inter = []
        for j in i:
            inter.append(E[j,:])
        fin.append(torch.stack(inter))
    
    
    
    
        
   
    return torch.stack(fin), torch.tensor(prepended_inp), attn_mask


In [ ]:
tokenizer1.encode(tokenizer1.bos_token)*7

In [ ]:
from sklearn.metrics import *

In [ ]:
def get_performance_test():
    torch.use_deterministic_algorithms(mode=True)
    set_seed(42)
    tinymodel.eval()
    model1.eval()
    #set_seed(42)
    all_labs = []
    all_labs_clf = []
    counter = 0
    for ii,ti, gol, ids in test_dataset:

        #     print(0/0)
        ti = ti.unsqueeze(0)
        ii = ii.unsqueeze(0)
        gol = [gol]
        ids = [ids]
        # print(batch_model1, batch_model1)

        ids = list(map(lambda x: x.split('/')[1], ids))
        #         ids_ = []
        #         for i in range(len(ids)):
        #             kbs = ids[i].split('[KB]')
        #             caps = ids[i].split('[CAPTION]')[-1]
        #             try:
        #                 kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
        #             except IndexError as e:
        #                 kb = '[CAPTION] '+ caps
        #             ids_.append(kb)
        #         ids = ids_
        
        ids_ = []
        for i in range(len(ids)):
            kbs = ids[i].split('[KB]')
            #print('kbs ', kbs)
            caps = ids[i].split('[CAPTION]')[-1]
            #print(caps)
            kb = ''
            if len(kbs)>3:
                kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
            elif len(kbs)==3:
                #print('in len3')
                kb = '[KB] '+kbs[1]
                caps = kbs[2].split('[CAPTION]')
                kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
            elif len(kbs)==2:
                #print('in len 1')
                caps = kbs[1].split('[CAPTION]')
                kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
            elif len(kbs)==1:
                kb = kbs[0].strip()
                #print('int ', kb)



            ids_.append(kb)
        
        ids = ids_
        
        
        
        #         print(ids)
        texts = [id2text[i] for i in ids]
        ids = [kb_fb[i] for i in ids]

        #     prpmt = []
        #     for i,j in zip(ids,texts):
        #         prpmt.append(i+'. The meme text reads :'+j + '. Classifier thinks the meme is')



        i = ii.float().to(device)
        t = ti.float().to(device)
        batch_size = i.shape[0]
        with torch.no_grad():
            logits, m = tinymodel(i,t)
        # print(m)
        m = m.unsqueeze(dim=1)
        all_labs_clf.append(logits.argmax(dim=-1)[0].item())
        #         prmpt0, lab0 = get_tokens([tokenizer1.bos_token]*batch_size, begin=True)

        #         print(prmpt0.shape)

        gumbel_logits = F.gumbel_softmax(logits, tau=1, hard=True)
        # print(gumbel_logits)
        agmx = gumbel_logits.argmax(dim=1)

        # print(agmx, logits.argmax(dim=1))
        #         prpmt = ['model thinks the meme is ']*batch_size

        #portion1,lab1, a1 = get_tokens(prpmt)

        portion2,lab3, _ = get_tokens(['output of the classifier multimodal embedding is ']*batch_size)
        lab4 = [tokenizer1.encode('-')[:] for i in range(batch_size)]
        lab4 = torch.tensor(lab4)

        # portion3 = get_tokens(['the meme is actually'])






        kk = []
        labs_verbalized = []
        # batch_prompt = ['the model thinks the meme is ']*32
        lab2 = []
        for i in gumbel_logits:
            agmx = i.argmax()
            # print(agmx)
            # print(dix[agmx.item()])
            tokenized_ = tokenizer1.encode(dix[agmx.item()])[:]
            labs_verbalized.append(dix[agmx.item()])
            # print(tokenized_)
            lab2.append(tokenized_)
            embedding = E[tokenized_[0]]
            one_hot = i.view(-1, 1)
            # print(embedding.shape, one_hot, one_hot.shape)

            e = torch.sum(one_hot*embedding.to(device), dim=0)
            kk.append(e)
        lab2 = torch.tensor(lab2)
        prpmt = []
        for i,j,z in zip(ids,texts,labs_verbalized):
            prpmt.append(i+'. The meme text reads :'+j + '. Classifier thinks the meme is {}'.format(z))
            #all_labs_clf.append(z)
        portion1,lab1, a1 = get_tokens(prpmt)
        string = ['the meme is actually']
        portion3,lab5,_ = get_tokens(string)
        inp_embed = torch.stack(kk).unsqueeze(dim=1)
        # print(inp_embed)
        # print(inp_embed.shape)
        # print(portion1.shape,inp_embed.shape,portion2.shape,m.shape,portion3.shape)

        final_embeds = torch.cat((portion1,inp_embed,portion2,m,portion3),dim=1)
        # print(final_embeds.shape)
        # print(lab1.shape, lab2.shape, lab3.shape, lab4.shape, lab5.shape)
        labs_shape = torch.cat((lab2,lab3,lab4,lab5),dim=1).shape
        remaining_mask = torch.ones(labs_shape[0], labs_shape[1]).long()
        #print(a1)
        #print(remaining_mask)

        full_mask = torch.cat((a1,remaining_mask),dim=1)
        #print('fm ', full_mask.shape)
        labs = torch.cat((lab1,lab2,lab3,lab4,lab5),dim=1)
        #print(full_mask)
        #print(0/0)
        #         print(final_embeds.shape, labs.shape)
        #         print(0/0)
        with torch.no_grad():
            output2 = model1(inputs_embeds = final_embeds.float(), labels = labs.long(), attention_mask=full_mask.to('cuda'))
        # print(output2.loss)
        # print(0/0)
        loss_lm = output2.loss
        #print(tokenizer1.batch_decode(output2.logits.argmax(dim=-1)))

        #print(loss_lm)
        llm_lab = tokenizer1.decode(output2.logits.argmax(dim=-1)[0][-1])
        all_labs.append(llm_lab)
        counter+=1
        #if counter==10:
        #    break
        
    return all_labs, all_labs_clf








In [ ]:
def get_performance_test_nm():
    torch.use_deterministic_algorithms(mode=True)
    set_seed(42)
    tinymodel.eval()
    model1.eval()
    #set_seed(42)
    all_labs = []
    all_labs_clf = []
    counter = 0
    for ii,ti, gol, ids in test_dataset:

        #     print(0/0)
        ti = ti.unsqueeze(0)
        ii = ii.unsqueeze(0)
        gol = [gol]
        ids = [ids]
        # print(batch_model1, batch_model1)

        ids = list(map(lambda x: x.split('/')[1], ids))
        #         print(ids)
        texts = [id2text[i] for i in ids]
        ids = [kb_fb[i] for i in ids]
        #         ids_ = []
        #         for i in range(len(ids)):
        #             kbs = ids[i].split('[KB]')
        #             caps = ids[i].split('[CAPTION]')[-1]
        #             try:
        #                 kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
        #             except IndexError as e:
        #                 kb = '[CAPTION] '+ caps
        #             ids_.append(kb)
        #         ids = ids_
        
        
        ids_ = []
        for i in range(len(ids)):
            kbs = ids[i].split('[KB]')
            #print('kbs ', kbs)
            caps = ids[i].split('[CAPTION]')[-1]
            #print(caps)
            kb = ''
            if len(kbs)>3:
                kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
            elif len(kbs)==3:
                #print('in len3')
                kb = '[KB] '+kbs[1]
                caps = kbs[2].split('[CAPTION]')
                kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
            elif len(kbs)==2:
                #print('in len 1')
                caps = kbs[1].split('[CAPTION]')
                kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
            elif len(kbs)==1:
                kb = kbs[0].strip()
                #print('int ', kb)



            ids_.append(kb)
        
        ids = ids_

        #     prpmt = []
        #     for i,j in zip(ids,texts):
        #         prpmt.append(i+'. The meme text reads :'+j + '. Classifier thinks the meme is')



        i = ii.float().to(device)
        t = ti.float().to(device)
        batch_size = i.shape[0]
        with torch.no_grad():
            logits, m = tinymodel(i,t)
        # print(m)
        m = m.unsqueeze(dim=1)
        #         prmpt0, lab0 = get_tokens([tokenizer1.bos_token]*batch_size, begin=True)

        #         print(prmpt0.shape)

        gumbel_logits = F.gumbel_softmax(logits, tau=1, hard=True)
        # print(gumbel_logits)
        agmx = gumbel_logits.argmax(dim=1)

        # print(agmx, logits.argmax(dim=1))
        #         prpmt = ['model thinks the meme is ']*batch_size

        #portion1,lab1, a1 = get_tokens(prpmt)


        # portion3 = get_tokens(['the meme is actually'])






        kk = []
        labs_verbalized = []
        # batch_prompt = ['the model thinks the meme is ']*32
        lab2 = []
        for i in gumbel_logits:
            agmx = i.argmax()
            # print(agmx)
            # print(dix[agmx.item()])
            tokenized_ = tokenizer1.encode(dix[agmx.item()])[:]
            labs_verbalized.append(dix[agmx.item()])
            # print(tokenized_)
            lab2.append(tokenized_)
            embedding = E[tokenized_[0]]
            one_hot = i.view(-1, 1)
            # print(embedding.shape, one_hot, one_hot.shape)

            e = torch.sum(one_hot*embedding.to(device), dim=0)
            kk.append(e)
        lab2 = torch.tensor(lab2)
        prpmt = []
        for i,j,z in zip(ids,texts,labs_verbalized):
            prpmt.append(i+'. The meme text reads :'+j)
            all_labs_clf.append(z)
        portion1,lab1, a1 = get_tokens(prpmt)
        string = ['the meme is actually']
        portion3,lab5,_ = get_tokens(string)
        inp_embed = torch.stack(kk).unsqueeze(dim=1)
        # print(inp_embed)
        # print(inp_embed.shape)
        # print(portion1.shape,inp_embed.shape,portion2.shape,m.shape,portion3.shape)

        final_embeds = torch.cat((portion1,portion3),dim=1)
        # print(final_embeds.shape)
        # print(lab1.shape, lab2.shape, lab3.shape, lab4.shape, lab5.shape)
        labs_shape = lab5.shape
        remaining_mask = torch.ones(labs_shape[0], labs_shape[1]).long()
        #print(a1)
        #print(remaining_mask)

        full_mask = torch.cat((a1,remaining_mask),dim=1)
        #print('fm ', full_mask.shape)
        labs = torch.cat((lab1,lab5),dim=1)
        #print(full_mask)
        #print(0/0)
        #         print(final_embeds.shape, labs.shape)
        #         print(0/0)
        with torch.no_grad():
            output2 = model1(inputs_embeds = final_embeds.float(), labels = labs.long(), attention_mask=full_mask.to('cuda'))
        # print(output2.loss)
        # print(0/0)
        loss_lm = output2.loss
        #print(tokenizer1.batch_decode(output2.logits.argmax(dim=-1)))

        #print(loss_lm)
        llm_lab = tokenizer1.decode(output2.logits.argmax(dim=-1)[0][-1])
        all_labs.append(llm_lab)
        counter+=1
        
    return all_labs, all_labs_clf








In [ ]:
gl_test  = []
for _,_,g,_ in test_dataset:
    gl_test.append(g)

In [ ]:

# # kbs = ids[0].split('[KB]')
# # print(kbs)
# # caps = ids[0].split('[CAPTION]')[-1]
# # print(caps)
# # if len(kbs)>3:
# #     kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
# # elif len(kbs)==3:
# #     kb = '[KB] '+kbs[1]
# #     caps = kb[2].split('[CAPTION]')[-1]
# #     kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps
# # elif len(kbs)==2:
# #     caps = kb[1].split('[CAPTION]')[-1]
# #     kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps
# # elif len(kbs)==1:
# #     print('int ', kb)
    
    
# # print(kb)


# for ii,ti, gol, ids in tqdm(dataloader):
#     ids = list(map(lambda x: x.split('/')[1], ids))
#     ids = [kb_fb[i] for i in ids]
#     #print(ids)
#     ids_ = []
#     for i in range(len(ids)):
#         kbs = ids[i].split('[KB]')
#         #print('kbs ', kbs)
#         caps = ids[i].split('[CAPTION]')[-1]
#         #print(caps)
#         kb = ''
#         if len(kbs)>3:
#             kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
#         elif len(kbs)==3:
#             #print('in len3')
#             kb = '[KB] '+kbs[1]
#             caps = kbs[2].split('[CAPTION]')
#             kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
#         elif len(kbs)==2:
#             #print('in len 1')
#             caps = kbs[1].split('[CAPTION]')
#             kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
#         elif len(kbs)==1:
#             kb = kbs[0].strip()
#             #print('int ', kb)


            
#         ids_.append(kb)
        
#     break


In [ ]:
from sklearn.metrics import *

In [ ]:
import torch
#os.environ['CUBLAS_WORKSPACE_CONFIG']=:4096:8
torch.use_deterministic_algorithms(True)
set_seed(42)
import numpy as np
model1_name = 'gpt2'
tokenizer1 = GPT2Tokenizer.from_pretrained(model1_name)
model1 = GPT2LMHeadModel.from_pretrained(model1_name,output_hidden_states=True).to(device)
mdl = fusion(512,512,True,256,512,0.1).to(device)
mdl_rand = fusion(512,512,True,256,512,0.1).to(device)
tinymodel = TinyModel(mdl,mdl_rand,rand=False).to(device)
E = model1.transformer.wte.weight.detach()
# E = model1.model.model.embed_tokens.weight.detach()
optimizer = optim.Adam(tinymodel.parameters(), lr=0.0005)
optimizer1 = optim.Adam(model1.parameters(), lr=0.0005)
dix = {0:'normal', 1:'offensive'}

tinymodel.train()
model1.train()
# Training loop
num_epochs = 2
def transpose(x):
    return x.transpose(-2, -1)

for epoch in tqdm(range(num_epochs)):
    #     set_seed(42)
    l=0
    cnt = 0
    l_lm = 0
    l_clf = 0
    tinymodel.train()
    model1.train()
    for ii,ti, gol, ids in dataloader:
        # print(batch_model1, batch_model1)
       
        ids = list(map(lambda x: x.split('/')[1], ids))
        #         print(ids)
        texts = [id2text[i] for i in ids]
        ids = [kb_fb[i] for i in ids]
        
        #         ids_ = []
        #         for i in range(len(ids)):
        #             kbs = ids[i].split('[KB]')
        #             caps = ids[i].split('[CAPTION]')[-1]
        #             try:
        #                 kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
        #             except IndexError as e:
        #                 kb = '[CAPTION] '+ caps
        #             ids_.append(kb)
        #         ids = ids_

        ids_ = []
        for i in range(len(ids)):
            kbs = ids[i].split('[KB]')
            #print('kbs ', kbs)
            caps = ids[i].split('[CAPTION]')[-1]
            #print(caps)
            kb = ''
            if len(kbs)>3:
                kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
            elif len(kbs)==3:
                #print('in len3')
                kb = '[KB] '+kbs[1]
                caps = kbs[2].split('[CAPTION]')
                kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
            elif len(kbs)==2:
                #print('in len 1')
                caps = kbs[1].split('[CAPTION]')
                kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
            elif len(kbs)==1:
                kb = kbs[0].strip()
                #print('int ', kb)



            ids_.append(kb)
        
        ids = ids_
        
        
        
       
        
        i = ii.float().to(device)
        t = ti.float().to(device)
        batch_size = i.shape[0]
        optimizer.zero_grad()
        optimizer1.zero_grad()
        logits, m = tinymodel(i,t)
        # print(m)
        m = m.unsqueeze(dim=1)
        #         prmpt0, lab0 = get_tokens([tokenizer1.bos_token]*batch_size, begin=True)
       
        #         print(prmpt0.shape)

        gumbel_logits = F.gumbel_softmax(logits, tau=1, hard=True)
        # print(gumbel_logits)
        agmx = gumbel_logits.argmax(dim=1)

        # print(agmx, logits.argmax(dim=1))
        #         prpmt = ['model thinks the meme is ']*batch_size

        
      
        portion2,lab3, _ = get_tokens(['output of the classifier multimodal embedding is ']*batch_size)
        lab4 = [tokenizer1.encode('-')[:] for i in range(batch_size)]
        lab4 = torch.tensor(lab4)

        # portion3 = get_tokens(['the meme is actually'])






        kk = []
        labs_verbalized = []
        # batch_prompt = ['the model thinks the meme is ']*32
        lab2 = []
        for i in gumbel_logits:
            agmx = i.argmax()
            # print(agmx)
            # print(dix[agmx.item()])
            tokenized_ = tokenizer1.encode(dix[agmx.item()])[:]
            labs_verbalized.append(dix[agmx.item()])
            # print(tokenized_)
            lab2.append(tokenized_)
            embedding = E[tokenized_[0]]
            one_hot = i.view(-1, 1)
            # print(embedding.shape, one_hot, one_hot.shape)

            e = torch.sum(one_hot*embedding.to(device), dim=0)
            kk.append(e)
        lab2 = torch.tensor(lab2)
        #string = ['the meme is actually {}'.format(i) for i in labs_verbalized] # why this fucking mistake??????
        
        prpmt = []
        for i,j,z in zip(ids,texts,labs_verbalized):
            prpmt.append(i+'. The meme text reads :'+j + '. Classifier thinks the meme is {}'.format(z))
        portion1,lab1, a1 = get_tokens(prpmt)
        
        string = ['the meme is actually {}'.format(dix[int(i)]) for i in gol]
        portion3,lab5,_ = get_tokens(string)
        inp_embed = torch.stack(kk).unsqueeze(dim=1)
        # print(inp_embed)
        # print(inp_embed.shape)
        # print(portion1.shape,inp_embed.shape,portion2.shape,m.shape,portion3.shape)
       
        final_embeds = torch.cat((portion1,inp_embed,portion2,m,portion3),dim=1)
        # print(final_embeds.shape)
        # print(lab1.shape, lab2.shape, lab3.shape, lab4.shape, lab5.shape)
        labs_shape = torch.cat((lab2,lab3,lab4,lab5),dim=1).shape
        remaining_mask = torch.ones(labs_shape[0], labs_shape[1]).long()
        #print(a1)
        #print(remaining_mask)
        
        full_mask = torch.cat((a1,remaining_mask),dim=1)
        #print('fm ', full_mask.shape)
        labs = torch.cat((lab1,lab2,lab3,lab4,lab5),dim=1)
        #print(full_mask)
        #print(0/0)
        #         print(final_embeds.shape, labs.shape)
        #         print(0/0)
        output2 = model1(inputs_embeds = final_embeds.float(), labels = labs.long(), attention_mask=full_mask.to('cuda'))
        # print(output2.loss)
        # print(0/0)
        loss_lm = output2.loss




        loss_clf = torch.nn.functional.cross_entropy(logits, gol.to(device), reduction='mean')

        #print(loss_lm, loss_clf)
       
        loss = loss_lm+loss_clf

        loss.backward()
        optimizer.step()
        optimizer1.step()


        l+=loss.detach().item()
        l_clf+=loss_clf.detach().item()
        l_lm+=loss_lm.detach().item()
        cnt+=1

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {l/cnt}, Loss clf: {l_clf/cnt}, Loss llm: {l_lm/cnt}")
    tinymodel.eval()
    pred,_ = tinymodel(im_tensor_.float().to(device),tx_tensor_.float().to(device))
    pred = pred.argmax(dim=1).detach().cpu().numpy()
    print(f1_score(gl_, pred, average='macro'), accuracy_score(gl_, pred))
    al, alc = get_performance_test()
    l2l = {'normal':0, 'offensive':1}
    al_ = list(map(lambda x: l2l[x.strip()], al))
    #alc_ = list(map(lambda x: l2l[x.strip()], alc))
    print(f1_score(gl_test, al_, average='macro'), f1_score(gl_test, alc, average='macro'))
    print(accuracy_score(gl_test, al_), accuracy_score(gl_test, alc))
    
    
print("Training complete!")


In [ ]:
#torch.save(model1.state_dict(), 'gpt2_backbone.pt')

In [ ]:
#torch.save(tinymodel.state_dict(), 'classifier.pt')

In [ ]:
import torch
device = 'cuda'
#os.environ['CUBLAS_WORKSPACE_CONFIG']=:4096:8
torch.use_deterministic_algorithms(True)
set_seed(42)
import numpy as np
model1_name = 'gpt2'
tokenizer1 = GPT2Tokenizer.from_pretrained(model1_name)
model1 = GPT2LMHeadModel.from_pretrained(model1_name,output_hidden_states=True).to(device)
mdl = fusion(512,512,True,256,512,0.1).to(device)
mdl_rand = fusion(512,512,True,256,512,0.1).to(device)
tinymodel = TinyModel(mdl,mdl_rand,rand=False).to(device)

In [ ]:
E = model1.transformer.wte.weight.detach()
model1.eval()
tinymodel.eval()
dix = {0:'normal', 1:'offensive'}
al, alc = get_performance_test()
l2l = {'normal':0, 'offensive':1}

al_ = list(map(lambda x: l2l[x.strip()], al))
print(f1_score(gl_test, al_, average='macro'), f1_score(gl_test, alc, average='macro'))
print(accuracy_score(gl_test, al_), accuracy_score(gl_test, alc))

In [ ]:
l2l = {'normal':0, 'a':1}

al_ = list(map(lambda x: l2l[x.strip()], al))
print(f1_score(gl_test, al_, average='macro'), f1_score(gl_test, alc, average='macro'))
print(accuracy_score(gl_test, al_), accuracy_score(gl_test, alc))

In [ ]:
!pip install captum

In [ ]:
PAD_IND = tokenizer1.eos_token_id

In [ ]:
PAD_IND

In [ ]:
model1.transformer.wte

In [ ]:

from captum.attr import IntegratedGradients, TokenReferenceBase, LayerIntegratedGradients

# Define the input text you want to analyze
input_text = "Your input text here"
token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
# Tokenize the input text and convert it to a tensor
input_ids = tokenizer1.encode(input_text, return_tensors="pt").to(torch.int64).to('cuda')
print(input_ids)
# Generate the output predictions

def squad_pos_forward_func(inputs):
    pred = model1(inputs)
    
    pred = pred.logits[:,-1,:].max(1)
    print(pred.values)
    return pred.values

squad_pos_forward_func(input_ids)

# with torch.no_grad():
#     output = model1(input_ids)

# Define a target position for attribution (e.g., the last token)
target_position = -1  # Adjust this based on your use case

# Initialize Integrated Gradients
#ig = IntegratedGradients(model1)
#ig = IntegratedGradients(model1.transformer.wte)
#ig = LayerIntegratedGradients(model1, model1.transformer.wte)
ig = LayerIntegratedGradients(squad_pos_forward_func, model1.transformer.wte)


In [ ]:
seq_length = 4
# Compute feature attribution scores
reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)
attributions, delta = ig.attribute(inputs = input_ids.to(torch.int64), baselines = reference_indices.to(torch.int64), return_convergence_delta=True)

# Convert attributions tensor to a numpy array
attributions = attributions.sum(dim=-1).squeeze(0).cpu().numpy()

# Normalize the attributions
attributions = (attributions - attributions.min()) / (attributions.max() - attributions.min())

# Map the attributions back to tokens
tokens = tokenizer1.convert_ids_to_tokens(input_ids[0].tolist())

# Display the feature attributions for each token
for token, attribution in zip(tokens, attributions):
    print(f"{token}: {attribution:.3f}")


In [ ]:
def fwd_model(inputs, labels, attention_mask):
    #print(tokenizer1.batch_decode(labels))
    #pred = model1(inputs_embeds = inputs, labels = labels, attention_mask=attention_mask)
    pred = model1(input_ids = labels, labels = labels, attention_mask=attention_mask)
    pred = pred.logits[:,-1,:].max(1)
    #print(pred.values)
    return pred.values

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
def get_performance_test():
    torch.use_deterministic_algorithms(mode=True)
    set_seed(42)
    tinymodel.eval()
    model1.eval()
    #set_seed(42)
    all_labs = []
    all_labs_clf = []
    counter = 0
    for ii,ti, gol, ids in test_dataset:

        #     print(0/0)
        ti = ti.unsqueeze(0)
        ii = ii.unsqueeze(0)
        gol = [gol]
        ids = [ids]
        # print(batch_model1, batch_model1)

        ids = list(map(lambda x: x.split('/')[1], ids))
        #         ids_ = []
        #         for i in range(len(ids)):
        #             kbs = ids[i].split('[KB]')
        #             caps = ids[i].split('[CAPTION]')[-1]
        #             try:
        #                 kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
        #             except IndexError as e:
        #                 kb = '[CAPTION] '+ caps
        #             ids_.append(kb)
        #         ids = ids_
        
        ids_ = []
        texts = [id2text[i] for i in ids]
        ids = [kb_fb[i] for i in ids]
        for i in range(len(ids)):
            kbs = ids[i].split('[KB]')
            print('kbs ', kbs)
            caps = ids[i].split('[CAPTION]')[-1]
            #print(caps)
            kb = ''
            if len(kbs)>3:
                print('in len >3')
                kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
            elif len(kbs)==3:
                print('in len3')
                kb = '[KB] '+kbs[1]
                caps = kbs[2].split('[CAPTION]')
                kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
            elif len(kbs)==2:
                print('in len 1')
                caps = kbs[1].split('[CAPTION]')
                kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
            elif len(kbs)==1:
                print('in len1')
                kb = kbs[0].strip()
                #print('int ', kb)



            ids_.append(kb)
        
        ids = ids_
        
        
        
        print('ids',ids)
        
        
        
        #     prpmt = []
        #     for i,j in zip(ids,texts):
        #         prpmt.append(i+'. The meme text reads :'+j + '. Classifier thinks the meme is')



        i = ii.float().to(device)
        t = ti.float().to(device)
        batch_size = i.shape[0]
        #with torch.no_grad():
        logits, m = tinymodel(i,t)
        # print(m)
        m = m.unsqueeze(dim=1)
        all_labs_clf.append(logits.argmax(dim=-1)[0].item())
        #         prmpt0, lab0 = get_tokens([tokenizer1.bos_token]*batch_size, begin=True)

        #         print(prmpt0.shape)

        gumbel_logits = F.gumbel_softmax(logits, tau=1, hard=True)
        # print(gumbel_logits)
        agmx = gumbel_logits.argmax(dim=1)

        # print(agmx, logits.argmax(dim=1))
        #         prpmt = ['model thinks the meme is ']*batch_size

        #portion1,lab1, a1 = get_tokens(prpmt)

        portion2,lab3, _ = get_tokens(['output of the classifier multimodal embedding is ']*batch_size)
        lab4 = [tokenizer1.encode('-')[:] for i in range(batch_size)]
        lab4 = torch.tensor(lab4)

        # portion3 = get_tokens(['the meme is actually'])






        kk = []
        labs_verbalized = []
        # batch_prompt = ['the model thinks the meme is ']*32
        lab2 = []
        for i in gumbel_logits:
            agmx = i.argmax()
            # print(agmx)
            # print(dix[agmx.item()])
            tokenized_ = tokenizer1.encode(dix[agmx.item()])[:]
            labs_verbalized.append(dix[agmx.item()])
            # print(tokenized_)
            lab2.append(tokenized_)
            embedding = E[tokenized_[0]]
            one_hot = i.view(-1, 1)
            # print(embedding.shape, one_hot, one_hot.shape)

            e = torch.sum(one_hot*embedding.to(device), dim=0)
            kk.append(e)
        lab2 = torch.tensor(lab2)
        prpmt = []
        for i,j,z in zip(ids,texts,labs_verbalized):
            prpmt.append(i+'. The meme text reads :'+j + '. Classifier thinks the meme is {}'.format(z))
            #all_labs_clf.append(z)
        portion1,lab1, a1 = get_tokens(prpmt)
        string = ['the meme is actually']
        portion3,lab5,_ = get_tokens(string)
        inp_embed = torch.stack(kk).unsqueeze(dim=1)
        # print(inp_embed)
        # print(inp_embed.shape)
        # print(portion1.shape,inp_embed.shape,portion2.shape,m.shape,portion3.shape)

        final_embeds = torch.cat((portion1,inp_embed,portion2,m,portion3),dim=1)
        # print(final_embeds.shape)
        # print(lab1.shape, lab2.shape, lab3.shape, lab4.shape, lab5.shape)
        labs_shape = torch.cat((lab2,lab3,lab4,lab5),dim=1).shape
        remaining_mask = torch.ones(labs_shape[0], labs_shape[1]).long()
        #print(a1)
        #print(remaining_mask)

        full_mask = torch.cat((a1,remaining_mask),dim=1)
        #print('fm ', full_mask.shape)
        labs = torch.cat((lab1,lab2,lab3,lab4,lab5),dim=1)
        #print(full_mask)
        #print(0/0)
        #         print(final_embeds.shape, labs.shape)
        #         print(0/0)
        ig = LayerIntegratedGradients(fwd_model, model1.transformer.wte)
        attributions, delta_start = ig.attribute(inputs=labs.long().to('cuda'),
                                  additional_forward_args=(labs.long().to('cuda'), full_mask.to('cuda')),
                                  return_convergence_delta=True)
        
        attributions = attributions.sum(dim=-1).squeeze(0).cpu().numpy()
        print(attributions)
        print(attributions.max())
        print(attributions.min())
        # Normalize the attributions
        attributions = (attributions - attributions.min()) / (attributions.max() - attributions.min())
        print(attributions)
        
        print(0/0)
            
        with torch.no_grad():
            output2 = model1(inputs_embeds = final_embeds.float(), labels = labs.long(), attention_mask=full_mask.to('cuda'))
        # print(output2.loss)
        # print(0/0)
        loss_lm = output2.loss
        #print(tokenizer1.batch_decode(output2.logits.argmax(dim=-1)))

        #print(loss_lm)
        llm_lab = tokenizer1.decode(output2.logits.argmax(dim=-1)[0][-1])
        all_labs.append(llm_lab)
        counter+=1
        #if counter==10:
        #    break
        
    return all_labs, all_labs_clf








In [ ]:
get_performance_test()

In [ ]:
50257*4/2

In [ ]:
model1.load_state_dict(torch.load('/kaggle/input/models-explanation/gpt2_backbone.pt'))
tinymodel.load_state_dict(torch.load('/kaggle/input/models-explanation/classifier (1).pt'))

In [ ]:
!cp /kaggle/working/classifier.pt /kaggle/input/kb-fb-ds

In [ ]:
# import torch
# #os.environ['CUBLAS_WORKSPACE_CONFIG']=:4096:8
# torch.use_deterministic_algorithms(True)
# set_seed(42)
# import numpy as np
# model1_name = 'gpt2'
# tokenizer1 = GPT2Tokenizer.from_pretrained(model1_name)
# model1 = GPT2LMHeadModel.from_pretrained(model1_name,output_hidden_states=True).to(device)
# mdl_rand = fusion(512,512,True,256,512,0.1).to(device)
# tinymodel = TinyModel(mdl,mdl_rand,rand=False).to(device)
# E = model1.transformer.wte.weight.detach()
# # E = model1.model.model.embed_tokens.weight.detach()
# optimizer = optim.Adam(tinymodel.parameters(), lr=0.0005)
# optimizer1 = optim.Adam(model1.parameters(), lr=0.0005)
# dix = {0:'normal', 1:'offensive'}

# tinymodel.train()
# model1.train()
# # Training loop
# num_epochs = 2
# def transpose(x):
#     return x.transpose(-2, -1)

# for epoch in tqdm(range(num_epochs)):
#     #     set_seed(42)
#     l=0
#     cnt = 0
#     l_lm = 0
#     l_clf = 0
#     tinymodel.train()
#     model1.train()
#     for ii,ti, gol, ids in dataloader:
#         # print(batch_model1, batch_model1)
       
#         ids = list(map(lambda x: x.split('/')[1], ids))
#         #         print(ids)
#         texts = [id2text[i] for i in ids]
#         ids = [kb_fb[i] for i in ids]
        
#         #         ids_ = []
#         #         for i in range(len(ids)):
#         #             kbs = ids[i].split('[KB]')
#         #             caps = ids[i].split('[CAPTION]')[-1]
#         #             try:
#         #                 kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
#         #             except IndexError as e:
#         #                 kb = '[CAPTION] '+ caps
#         #             ids_.append(kb)
#         #         ids = ids_

#         ids_ = []
#         for i in range(len(ids)):
#             kbs = ids[i].split('[KB]')
#             #print('kbs ', kbs)
#             caps = ids[i].split('[CAPTION]')[-1]
#             #print(caps)
#             kb = ''
#             if len(kbs)>3:
#                 kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
#             elif len(kbs)==3:
#                 #print('in len3')
#                 kb = '[KB] '+kbs[1]
#                 caps = kbs[2].split('[CAPTION]')
#                 kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
#             elif len(kbs)==2:
#                 #print('in len 1')
#                 caps = kbs[1].split('[CAPTION]')
#                 kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
#             elif len(kbs)==1:
#                 kb = kbs[0].strip()
#                 #print('int ', kb)



#             ids_.append(kb)
        
#         ids = ids_
        
        
        
       
        
#         i = ii.float().to(device)
#         t = ti.float().to(device)
#         batch_size = i.shape[0]
#         optimizer.zero_grad()
#         optimizer1.zero_grad()
#         logits, m = tinymodel(i,t)
#         # print(m)
#         m = m.unsqueeze(dim=1)
#         #         prmpt0, lab0 = get_tokens([tokenizer1.bos_token]*batch_size, begin=True)
       
#         #         print(prmpt0.shape)

#         gumbel_logits = F.gumbel_softmax(logits, tau=1, hard=True)
#         # print(gumbel_logits)
#         agmx = gumbel_logits.argmax(dim=1)

#         # print(agmx, logits.argmax(dim=1))
#         #         prpmt = ['model thinks the meme is ']*batch_size

        
      
#         portion2,lab3, _ = get_tokens(['output of the classifier multimodal embedding is ']*batch_size)
#         lab4 = [tokenizer1.encode('-')[:] for i in range(batch_size)]
#         lab4 = torch.tensor(lab4)

#         # portion3 = get_tokens(['the meme is actually'])






#         kk = []
#         labs_verbalized = []
#         # batch_prompt = ['the model thinks the meme is ']*32
#         lab2 = []
#         for i in gumbel_logits:
#             agmx = i.argmax()
#             # print(agmx)
#             # print(dix[agmx.item()])
#             tokenized_ = tokenizer1.encode(dix[agmx.item()])[:]
#             labs_verbalized.append(dix[agmx.item()])
#             # print(tokenized_)
#             lab2.append(tokenized_)
#             embedding = E[tokenized_[0]]
#             one_hot = i.view(-1, 1)
#             # print(embedding.shape, one_hot, one_hot.shape)

#             e = torch.sum(one_hot*embedding.to(device), dim=0)
#             kk.append(e)
#         lab2 = torch.tensor(lab2)
#         #string = ['the meme is actually {}'.format(i) for i in labs_verbalized] # why this fucking mistake??????
        
#         prpmt = []
#         for i,j,z in zip(ids,texts,labs_verbalized):
#             prpmt.append(i+'. The meme text reads :'+j + '. Classifier thinks the meme is {}'.format(z))
#         portion1,lab1, a1 = get_tokens(prpmt)
        
#         string = ['the meme is actually {}'.format(dix[int(i)]) for i in gol]
#         portion3,lab5,_ = get_tokens(string)
#         inp_embed = torch.stack(kk).unsqueeze(dim=1)
#         # print(inp_embed)
#         # print(inp_embed.shape)
#         # print(portion1.shape,inp_embed.shape,portion2.shape,m.shape,portion3.shape)
       
#         final_embeds = torch.cat((portion1,inp_embed,portion2,m,portion3),dim=1)
#         # print(final_embeds.shape)
#         # print(lab1.shape, lab2.shape, lab3.shape, lab4.shape, lab5.shape)
#         labs_shape = torch.cat((lab2,lab3,lab4,lab5),dim=1).shape
#         remaining_mask = torch.ones(labs_shape[0], labs_shape[1]).long()
#         #print(a1)
#         #print(remaining_mask)
        
#         full_mask = torch.cat((a1,remaining_mask),dim=1)
#         #print('fm ', full_mask.shape)
#         labs = torch.cat((lab1,lab2,lab3,lab4,lab5),dim=1)
#         #print(full_mask)
#         #print(0/0)
#         #         print(final_embeds.shape, labs.shape)
#         #         print(0/0)
#         output2 = model1(inputs_embeds = final_embeds.float(), labels = labs.long(), attention_mask=full_mask.to('cuda'))
#         # print(output2.loss)
#         # print(0/0)
#         loss_lm = output2.loss




#         loss_clf = torch.nn.functional.cross_entropy(logits, gol.to(device), reduction='mean')

#         #print(loss_lm, loss_clf)
       
#         loss = loss_lm+loss_clf

#         loss.backward()
#         optimizer.step()
#         optimizer1.step()


#         l+=loss.detach().item()
#         l_clf+=loss_clf.detach().item()
#         l_lm+=loss_lm.detach().item()
#         cnt+=1

#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {l/cnt}, Loss clf: {l_clf/cnt}, Loss llm: {l_lm/cnt}")
#     tinymodel.eval()
#     pred,_ = tinymodel(im_tensor_.float().to(device),tx_tensor_.float().to(device))
#     pred = pred.argmax(dim=1).detach().cpu().numpy()
#     print(f1_score(gl_, pred, average='macro'), accuracy_score(gl_, pred))
#     al, alc = get_performance_test()
#     l2l = {'normal':0, 'offensive':1}
#     al_ = list(map(lambda x: l2l[x.strip()], al))
#     #alc_ = list(map(lambda x: l2l[x.strip()], alc))
#     print(f1_score(gl_test, al_, average='macro'), f1_score(gl_test, alc, average='macro'))
#     print(accuracy_score(gl_test, al_), accuracy_score(gl_test, alc))
    
    
# print("Training complete!")


In [ ]:
#E = model1.transformer.wte.weight.detach()

In [ ]:
# #all_labs
# import torch
# torch.manual_seed(42)
# import random
# random.seed(42)
# import numpy as np
# np.random.seed(42)
# tinymodel = TinyModel(mdl,rand=False).to(device)
# E = model1.transformer.wte.weight.detach()
# # E = model1.model.model.embed_tokens.weight.detach()
# optimizer = optim.Adam(tinymodel.parameters(), lr=0.0005)
# optimizer1 = optim.Adam(model1.parameters(), lr=0.0005)
# dix = {0:'normal', 1:'offensive'}

# tinymodel.train()
# model1.train()
# # Training loop
# num_epochs = 5
# def transpose(x):
#     return x.transpose(-2, -1)

# for epoch in range(num_epochs):
#     l=0
#     cnt = 0
#     l_lm = 0
#     l_clf = 0
#     tinymodel.train()
#     model1.train()
#     for ii,ti, gol, ids in tqdm(dataloader):
#         # print(batch_model1, batch_model1)
       
#         ids = list(map(lambda x: x.split('/')[1], ids))
#         #         print(ids)
#         texts = [id2text[i] for i in ids]
#         ids = [kb_fb[i] for i in ids]
        
#         ids_ = []
#         for i in range(len(ids)):
#             kbs = ids[i].split('[KB]')
#             #print('kbs ', kbs)
#             caps = ids[i].split('[CAPTION]')[-1]
#             #print(caps)
#             kb = ''
#             if len(kbs)>3:
#                 kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
#             elif len(kbs)==3:
#                 #print('in len3')
#                 kb = '[KB] '+kbs[1]
#                 caps = kbs[2].split('[CAPTION]')
#                 kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
#             elif len(kbs)==2:
#                 #print('in len 1')
#                 caps = kbs[1].split('[CAPTION]')
#                 kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
#             elif len(kbs)==1:
#                 kb = kbs[0].strip()
#                 #print('int ', kb)



#             ids_.append(kb)
        
#         ids = ids_
        
        
        
       
        
#         i = ii.float().to(device)
#         t = ti.float().to(device)
#         batch_size = i.shape[0]
#         optimizer.zero_grad()
#         optimizer1.zero_grad()
#         logits, m = tinymodel(i,t)
#         # print(m)
#         m = m.unsqueeze(dim=1)
#         #         prmpt0, lab0 = get_tokens([tokenizer1.bos_token]*batch_size, begin=True)
       
#         #         print(prmpt0.shape)

#         gumbel_logits = F.gumbel_softmax(logits, tau=1, hard=True)
#         # print(gumbel_logits)
#         agmx = gumbel_logits.argmax(dim=1)

#         # print(agmx, logits.argmax(dim=1))
#         #         prpmt = ['model thinks the meme is ']*batch_size

        
      
#         #         portion2,lab3, _ = get_tokens(['output of the classifier multimodal embedding is ']*batch_size)
#         #         lab4 = [tokenizer1.encode('-')[:] for i in range(batch_size)]
#         #         lab4 = torch.tensor(lab4)

#         # portion3 = get_tokens(['the meme is actually'])






#         kk = []
#         labs_verbalized = []
#         # batch_prompt = ['the model thinks the meme is ']*32
#         lab2 = []
#         for i in gumbel_logits:
#             agmx = i.argmax()
#             # print(agmx)
#             # print(dix[agmx.item()])
#             tokenized_ = tokenizer1.encode(dix[agmx.item()])[:]
#             labs_verbalized.append(dix[agmx.item()])
#             # print(tokenized_)
#             lab2.append(tokenized_)
#             embedding = E[tokenized_[0]]
#             one_hot = i.view(-1, 1)
#             # print(embedding.shape, one_hot, one_hot.shape)

#             e = torch.sum(one_hot*embedding.to(device), dim=0)
#             kk.append(e)
#         #         lab2 = torch.tensor(lab2)
#         #string = ['the meme is actually {}'.format(i) for i in labs_verbalized] # why this fucking mistake??????
        
#         prpmt = []
#         for i,j,z in zip(ids,texts,labs_verbalized):
#             prpmt.append(i+'. The meme text reads :'+j)
#         portion1,lab1, a1 = get_tokens(prpmt)
        
#         string = ['the meme is actually {}'.format(dix[int(i)]) for i in gol]
#         portion3,lab5,_ = get_tokens(string)
#         inp_embed = torch.stack(kk).unsqueeze(dim=1)
#         # print(inp_embed)
#         # print(inp_embed.shape)
#         # print(portion1.shape,inp_embed.shape,portion2.shape,m.shape,portion3.shape)
       
#         final_embeds = torch.cat((portion1,portion3),dim=1)
#         # print(final_embeds.shape)
#         # print(lab1.shape, lab2.shape, lab3.shape, lab4.shape, lab5.shape)
#         labs_shape = lab5.shape
#         remaining_mask = torch.ones(labs_shape[0], labs_shape[1]).long()
#         #print(a1)
#         #print(remaining_mask)
        
#         full_mask = torch.cat((a1,remaining_mask),dim=1)
#         #print('fm ', full_mask.shape)
#         labs = torch.cat((lab1,lab5),dim=1)
#         #print(full_mask)
#         #print(0/0)
#         #         print(final_embeds.shape, labs.shape)
#         #         print(0/0)
#         output2 = model1(inputs_embeds = final_embeds.float(), labels = labs.long(), attention_mask=full_mask.to('cuda'))
#         # print(output2.loss)
#         # print(0/0)
#         loss_lm = output2.loss




#         loss_clf = torch.nn.functional.cross_entropy(logits, gol.to(device), reduction='mean')

#         #print(loss_lm, loss_clf)
       
#         loss = loss_lm+loss_clf

#         loss.backward()
#         optimizer.step()
#         optimizer1.step()


#         l+=loss.detach().item()
#         l_clf+=loss_clf.detach().item()
#         l_lm+=loss_lm.detach().item()
#         cnt+=1

#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {l/cnt}, Loss clf: {l_clf/cnt}, Loss llm: {l_lm/cnt}")
#     tinymodel.eval()
#     pred,_ = tinymodel(im_tensor_.float().to(device),tx_tensor_.float().to(device))
#     pred = pred.argmax(dim=1).detach().cpu().numpy()
#     print(f1_score(gl_, pred, average='macro'), accuracy_score(gl_, pred))
#     al, alc = get_performance_test_nm()
#     l2l = {'normal':0, 'offensive':1}
#     al_ = list(map(lambda x: l2l[x.strip()], al))
#     alc_ = list(map(lambda x: l2l[x.strip()], alc))
#     print(f1_score(gl_test, al_, average='macro'), f1_score(gl_test, alc_, average='macro'))
#     print(accuracy_score(gl_test, al_), accuracy_score(gl_test, alc_))
    
    
# print("Training complete!")


In [ ]:
# from : https://github.com/huggingface/transformers/issues/6535
def prepare_inputs_for_generation(input_ids, past_key_values=None, **kwargs):
    token_type_ids = kwargs.get("token_type_ids", None)
    # only last token for inputs_ids if past_key_values is defined in kwargs
    if past_key_values:
        input_ids = input_ids[:, -1].unsqueeze(-1)
        if token_type_ids is not None:
            token_type_ids = token_type_ids[:, -1].unsqueeze(-1)

    attention_mask = kwargs.get("attention_mask", None)
    position_ids = kwargs.get("position_ids", None)

    if attention_mask is not None and position_ids is None:
        # create position_ids on the fly for batch generation
        position_ids = attention_mask.long().cumsum(-1) - 1
        position_ids.masked_fill_(attention_mask == 0, 1)
        if past_key_values:
            position_ids = position_ids[:, -1].unsqueeze(-1)
    else:
        position_ids = None

    # !!!!!!!!!!!!!!!!!!! start: modified vs original, to pass inputs_embeds when they are available
    if "inputs_embeds" in kwargs and past_key_values is None:  # we only want to use them in the 1st generation step
        model_inputs = {"inputs_embeds": inputs_embeds}
    else:
        model_inputs = {"input_ids": input_ids}
    model_inputs.update({
        "past_key_values": past_key_values,
        "use_cache": kwargs.get("use_cache"),
        "position_ids": position_ids,
        "attention_mask": attention_mask,
        "token_type_ids": token_type_ids,
    })
    return model_inputs
    # !!!!!!!!!!!!!!!!!!! end: modified vs original, to pass inputs_embeds when they are available

In [ ]:
inputs_embeds = model1.transformer.wte(torch.LongTensor([[787,32329,898]]).to(device))
print(inputs_embeds)

In [ ]:
from matplotlib.pyplot import imshow
from IPython.display import display

In [ ]:
import math

In [ ]:
!pip install shutup

In [ ]:
import shutup; shutup.please()

In [ ]:
d_ids

In [ ]:
set_seed(42)
start_index = 0
end_index = len(test_dataset)

# Specify how many random indices you want to select
num_indices_to_select = 100

# Use random.sample() to select random indices without duplicates
random_indices = random.sample(range(start_index, end_index + 1), num_indices_to_select)

print(random_indices)

test_dataset_1 = torch.utils.data.Subset(test_dataset, random_indices)

In [ ]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(42)
batch_size = 32
set_seed(42)

dataloader_test = DataLoader(
    torch.utils.data.Subset(test_dataset_1, [i for i in range(70)]),
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    worker_init_fn=seed_worker,
    generator=g,
)


    
    
    

In [ ]:
len(dataloader_test)

In [ ]:
d_ids = {}
def get_performance_test(second_pass=False, llm_label='', prev_prob = 0, prev_lab = '', counter=0):
    #print(test_dataset)
    torch.use_deterministic_algorithms(True)
    set_seed(42)
    
    if second_pass:
        #print('IN SECOND PASS')
        test_dataset_ = [test_dataset_1[counter]]
        
        
    else:
        test_dataset_ = test_dataset_1
    
    tinymodel.eval()
    model1.eval()
    all_labs = []
    all_labs_clf = []
    counter = 0
    
    for ii,ti, gol, ids in tqdm(test_dataset_,disable=second_pass):

        #     print(0/0)
        #print(ids)
        tmp_id = ids
        
        #         raw_image = Image.open('./data/'+ids)
        #         if not second_pass:
        #             display(raw_image)
        ti = ti.unsqueeze(0)
        ii = ii.unsqueeze(0)
        gol = [gol]
        ids = [ids]
        # print(batch_model1, batch_model1)

        ids = list(map(lambda x: x.split('/')[1], ids))
        #         ids_ = []
        #         for i in range(len(ids)):
        #             kbs = ids[i].split('[KB]')
        #             caps = ids[i].split('[CAPTION]')[-1]
        #             try:
        #                 kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
        #             except IndexError as e:
        #                 kb = '[CAPTION] '+ caps
        #             ids_.append(kb)
        #         ids = ids_
        
        ids_ = []
        for i in range(len(ids)):
            kbs = ids[i].split('[KB]')
            #print('kbs ', kbs)
            caps = ids[i].split('[CAPTION]')[-1]
            #print(caps)
            kb = ''
            if len(kbs)>3:
                kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
            elif len(kbs)==3:
                #print('in len3')
                kb = '[KB] '+kbs[1]
                caps = kbs[2].split('[CAPTION]')
                kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
            elif len(kbs)==2:
                #print('in len 1')
                caps = kbs[1].split('[CAPTION]')
                kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
            elif len(kbs)==1:
                kb = kbs[0].strip()
                #print('int ', kb)



            ids_.append(kb)
        
        ids = ids_
        
        
        
        #         print(ids)
        texts = [id2text[i] for i in ids]
        ids = [kb_fb[i] for i in ids]
        if second_pass:
            ids = llm_label
        #print('ids', ids)
        #print('text', texts)

        #     prpmt = []
        #     for i,j in zip(ids,texts):
        #         prpmt.append(i+'. The meme text reads :'+j + '. Classifier thinks the meme is')



        i = ii.float().to(device)
        t = ti.float().to(device)
        batch_size = i.shape[0]
        #with torch.no_grad():
        logits, m = tinymodel(i,t)
        # print(m)
        m = m.unsqueeze(dim=1)
        all_labs_clf.append(logits.argmax(dim=-1)[0].item())
        #         prmpt0, lab0 = get_tokens([tokenizer1.bos_token]*batch_size, begin=True)

        #         print(prmpt0.shape)

        gumbel_logits = F.gumbel_softmax(logits, tau=1, hard=True)
        # print(gumbel_logits)
        agmx = gumbel_logits.argmax(dim=1)

        # print(agmx, logits.argmax(dim=1))
        #         prpmt = ['model thinks the meme is ']*batch_size

        #portion1,lab1, a1 = get_tokens(prpmt)

        portion2,lab3, _ = get_tokens(['output of the classifier multimodal embedding is ']*batch_size)
        lab4 = [tokenizer1.encode('-')[:] for i in range(batch_size)]
        lab4 = torch.tensor(lab4)

        # portion3 = get_tokens(['the meme is actually'])






        kk = []
        labs_verbalized = []
        # batch_prompt = ['the model thinks the meme is ']*32
        lab2 = []
        for i in gumbel_logits:
            agmx = i.argmax()
            # print(agmx)
            # print(dix[agmx.item()])
            tokenized_ = tokenizer1.encode(dix[agmx.item()])[:]
            labs_verbalized.append(dix[agmx.item()])
            # print(tokenized_)
            lab2.append(tokenized_)
            embedding = E[tokenized_[0]]
            one_hot = i.view(-1, 1)
            # print(embedding.shape, one_hot, one_hot.shape)

            e = torch.sum(one_hot*embedding.to(device), dim=0)
            kk.append(e)
        lab2 = torch.tensor(lab2)
        prpmt = []
        for i,j,z in zip(ids,texts,labs_verbalized):
            prpmt.append(i+'. The meme text reads :'+j + '. Classifier thinks the meme is {}'.format(z))
            #all_labs_clf.append(z)
        portion1,lab1, a1 = get_tokens(prpmt)
        
        string = ['the meme is actually']
        #         if second_pass:
        #             string[0] += '{} because of'.format(llm_label)
        #             print(string)
        portion3,lab5,_ = get_tokens(string)
        inp_embed = torch.stack(kk).unsqueeze(dim=1)
        # print(inp_embed)
        # print(inp_embed.shape)
        # print(portion1.shape,inp_embed.shape,portion2.shape,m.shape,portion3.shape)

        final_embeds = torch.cat((portion1,inp_embed,portion2,m,portion3),dim=1)
        # print(final_embeds.shape)
        # print(lab1.shape, lab2.shape, lab3.shape, lab4.shape, lab5.shape)
        labs_shape = torch.cat((lab2,lab3,lab4,lab5),dim=1).shape
        remaining_mask = torch.ones(labs_shape[0], labs_shape[1]).long()
        #print(a1)
        #print(remaining_mask)

        full_mask = torch.cat((a1,remaining_mask),dim=1)
        #print('fm ', full_mask.shape)
        labs = torch.cat((lab1,lab2,lab3,lab4,lab5),dim=1)
        #print(full_mask)
        #print(0/0)
        #         print(final_embeds.shape, labs.shape)
        #         print(0/0)
        #with torch.no_grad():
        if second_pass:
            #model1.prepare_inputs_for_generation = prepare_inputs_for_generation
            #input_ids = torch.LongTensor([[model1.config.bos_token_id]]).to(device)
            #inputs_embdes = final_embeds.float().to(device)
            #op = model1.sample(input_ids, inputs_embeds = inputs_embdes, pad_token_id=model1.config.eos_token_id)
            #print(tokenizer1.batch_decode(op))
            output2 = model1(inputs_embeds = final_embeds.float(), labels = labs.long(), attention_mask=full_mask.to('cuda'))
            #print('SECOND PASS ',tokenizer1.batch_decode(output2.logits[:,-1,:].squeeze(dim=0).detach().cpu().topk(40).indices))
            second_pass_lab = tokenizer1.batch_decode(output2.logits[:,-1,:].squeeze(dim=0).detach().cpu().topk(40).indices)[0].strip()
            #print('pl', prev_lab)
            #print('sl', second_pass_lab)
            agmx_id = output2.logits.argmax(dim=-1)[0][-1].item()
            log_likelihood = output2.logits[:,-1,:]
            log_likelihood = torch.nn.functional.softmax(log_likelihood, dim=-1)[:,agmx_id][0].item()
            prev_prob = prev_prob[0].item()
            #print('current probability', log_likelihood)
            #print('previous probability', prev_prob)
            #             if (prev_lab==second_pass_lab) and (log_likelihood>=prev_prob):
            #if (prev_lab==second_pass_lab) and (abs(log_likelihood-prev_prob)<=0.05):
            #    return True
            
            if prev_lab==second_pass_lab:
                return True, log_likelihood
            
            else:
                return False, False
            
            
                                                      
        else:
            output2 = model1(inputs_embeds = final_embeds.float(), labels = labs.long(), attention_mask=full_mask.to('cuda'))
        # print(output2.loss)
        # print(0/0)
        loss_lm = output2.loss
        #print(tokenizer1.batch_decode(output2.logits.argmax(dim=-1)))

        #print(loss_lm)
        m.retain_grad()
        model1.transformer.wte.weight.retain_grad()
        #print(output2.logits.shape)
        #print(output2.logits.argmax(dim=-1)[0].shape)
        agmx_idx = output2.logits.argmax(dim=-1)[0][-1].item()
        #print(agmx_idx)
        #print(output2.logits[:,-1,:])
        log_likelihood_ = output2.logits[:,-1,:]
        log_likelihood = output2.logits[:,-1,agmx_idx]
        #print(log_likelihood)
        log_likelihood.backward()
        grad_  = m.grad.data.cpu().squeeze(1)
        #embedding_norm = model1.transformer.wte.weight.grad.data.cpu().norm(dim=-1)
        #print('en ', embedding_norm,embedding_norm.shape)
        #tokens_index = torch.mm(grad_, E.detach().cpu().T)[0]
        
        #tokens_index /= model1.transformer.wte.weight.detach().cpu().norm(dim=-1)
        #print(tokens_index)
        #ct_index = tokens_index*output2.logits[:,-1,:].squeeze(dim=0).detach().cpu() # ct_index = cumulative token index
        
        #print(tokens_index.topk(40).indices)
        #print(tokenizer1.batch_decode(embedding_norm.topk(40).indices))
        #print(tokenizer1.batch_decode(tokens_index.topk(40).indices))
        #print(tokenizer1.batch_decode(ct_index.topk(40).indices))
        #print(tokenizer1.batch_decode(output2.logits[:,-1,:].squeeze(dim=0).detach().cpu().topk(40).indices))
        #print(tokens_index.shape)
        #print(grad_.shape)
        ti,op,tt,t_id = grad_, output2.logits[:,-1,:].squeeze(dim=0).detach().cpu(), texts, tmp_id
        #print('****************************')
        #print(t_id)
        output_aware_token_idx = []
        output_aware_tokens = []
        for i in op.topk(3500).indices:
            k = model1.transformer.wte(i.cuda()).detach().cpu().unsqueeze(1)

            #output_aware_token_idx.append(ti.mm(k))
            val = torch.nn.functional.cosine_similarity(ti,k.T)[0].item()
             
            #if val>0:
            output_aware_token_idx.append(val)
            output_aware_tokens.append(tokenizer1.decode(i))
            #     print('{} -> {}'.format(tokenizer1.decode(i), ti.mm(k).squeeze()))
        
        median = np.median(output_aware_token_idx)
        p,q = [],[]
        #         for i,j in zip(output_aware_token_idx, output_aware_tokens):
        #             if i>=median:
        #                 p.append(i)
        #                 q.append(j)
                
        #         for i,j in zip(output_aware_token_idx, output_aware_tokens):
        #             if i>-0.1 and i<0.1:
        #                 p.append(i)
        #                 q.append(j)

        #         output_aware_token_idx = p
        #         output_aware_tokens = q
        
        output_aware_tokens = list(map(lambda x: x.strip().lower(), output_aware_tokens))
        
        image = preprocess(Image.open('./data/'+t_id)).unsqueeze(0).to(device)
        text = clip.tokenize(output_aware_tokens).to(device)
        # clip_feats = []
        text_cand = clip.tokenize(tt).to(device)



        with torch.no_grad():
            if_ = model.encode_image(image)
            tf_ = model.encode_text(text_cand)

            #print(if_.shape, tf_.shape)
            multimodal_features = if_
            text_features = model.encode_text(text)

            clip_feats = torch.nn.functional.cosine_similarity(multimodal_features, text_features, dim=1)


        clip_feats = clip_feats.cpu().numpy().tolist()
        
        combined_feats = []
        for i,j in zip(output_aware_token_idx, clip_feats):
            #tmp = 2*i*j/(i+j)
            tmp = 1.0*j+0.00*i
            #tmp = j
            combined_feats.append(tmp)
            
        final_tokens = []
        for i in np.argsort(combined_feats)[-1:-20:-1]:
            final_tokens.append(output_aware_tokens[i])
            #print(output_aware_tokens[i])
        
        final_tokens = list(dict.fromkeys(final_tokens))
        #final_tokens = list(final_tokens)
        s = ['when', 'why', 'our']
        final_tokens = [i for i in final_tokens if (i not in s and len(i)>2)][:20]
        
        op_preserving_kt = []
        
        for i in final_tokens:
            string = '[KB] {} are'.format(i)
            op = tokenizer1.batch_decode(model1.generate(torch.tensor(tokenizer1.encode(string)).unsqueeze(0).to('cuda'), max_length=14, pad_token_id=tokenizer1.eos_token_id))
            s = ''
            for i in op[0]:
                s += i
                if i=='.':
                    break
            op_preserving_kt.append(s)
        
        #print('op preserving ', op_preserving_kt)
        #print(texts[0],labs_verbalized[0])
        #print(ids[0])
        caption = ids[0].split('[CAPTION]')[-1]
        #print(caption)
        
        kb = ''
        for i in op_preserving_kt:
            kb+=i+' '
            
        #         kb1 += '[CAPTION] '+caption + 'output of the classifier multimodal embedding is '

        #         kb2 = '. The meme text reads :'+tt[0] + '. Classifier thinks the meme is {}'.format(labs_verbalized[0]))
        
        #print('****************************')
    
        
        
        
        op_preserving_final_tokens = []
        preservation_scores = []
        llm_lab = tokenizer1.decode(output2.logits.argmax(dim=-1)[0][-1])
        #print(llm_lab)
        for i in op_preserving_kt:
            kb = i+' '
            prev_prob = torch.nn.functional.softmax(log_likelihood_, dim=-1)[:,agmx_idx]
            #print(prev_prob)
            #break
            #[:,-1,agmx_idx]
            is_op_preserved,likelihood = get_performance_test(second_pass=True, llm_label=[kb], prev_lab=llm_lab.strip(), prev_prob=prev_prob, counter=counter)
            if is_op_preserved:
                preservation_scores.append(likelihood)
                op_preserving_final_tokens.append(i)
            
                
        #print('OP PRESERVING FINAL TOKENS ',op_preserving_final_tokens)      
        #print('PRESERVATION SCORES ',preservation_scores)
        toks = []
        for idx in list(reversed(list(np.argsort(preservation_scores))))[:4]:
            #print(idx)
            tmp = op_preserving_final_tokens[idx].split('are')[0][4:].strip()
            toks.append(tmp)
        
        #print('OP PRESERVING FINAL TOKENS ',toks)
        
        string = ''
        for i in toks:
            string+=i+'\t'
        
        #print(string)
        #print(string.split('\t'))
        
        d_ids[t_id] = string
        
        all_labs.append(llm_lab)
        counter+=1
    
        
        
    return all_labs, all_labs_clf, grad_, output2.logits[:,-1,:].squeeze(dim=0).detach().cpu(), texts, tmp_id








In [ ]:
set_seed(42)
al, alc,ti,op,tt,t_id = get_performance_test()

In [ ]:
E.shape

In [ ]:
d_ids = {}
def get_performance_test(second_pass=False, llm_label='', prev_prob = 0, prev_lab = '', counter=0):
    #print(test_dataset)
    torch.use_deterministic_algorithms(True)
    set_seed(42)
    
    if second_pass:
        #print('IN SECOND PASS')
        test_dataset_ = [test_dataset_1[counter]]
        
        
    else:
        test_dataset_ = test_dataset_1
    
    tinymodel.eval()
    model1.eval()
    all_labs = []
    all_labs_clf = []
    counter = 0
    
    for ii,ti, gol, ids in tqdm(test_dataset_,disable=second_pass):

        #     print(0/0)
        #print(ids)
        tmp_id = ids
        
        #         raw_image = Image.open('./data/'+ids)
        #         if not second_pass:
        #             display(raw_image)
        ti = ti.unsqueeze(0)
        ii = ii.unsqueeze(0)
        gol = [gol]
        ids = [ids]
        # print(batch_model1, batch_model1)

        ids = list(map(lambda x: x.split('/')[1], ids))
        #         ids_ = []
        #         for i in range(len(ids)):
        #             kbs = ids[i].split('[KB]')
        #             caps = ids[i].split('[CAPTION]')[-1]
        #             try:
        #                 kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
        #             except IndexError as e:
        #                 kb = '[CAPTION] '+ caps
        #             ids_.append(kb)
        #         ids = ids_
        
        ids_ = []
        for i in range(len(ids)):
            kbs = ids[i].split('[KB]')
            #print('kbs ', kbs)
            caps = ids[i].split('[CAPTION]')[-1]
            #print(caps)
            kb = ''
            if len(kbs)>3:
                kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
            elif len(kbs)==3:
                #print('in len3')
                kb = '[KB] '+kbs[1]
                caps = kbs[2].split('[CAPTION]')
                kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
            elif len(kbs)==2:
                #print('in len 1')
                caps = kbs[1].split('[CAPTION]')
                kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
            elif len(kbs)==1:
                kb = kbs[0].strip()
                #print('int ', kb)



            ids_.append(kb)
        
        ids = ids_
        
        
        
        #         print(ids)
        texts = [id2text[i] for i in ids]
        ids = [kb_fb[i] for i in ids]
        if second_pass:
            ids = llm_label
        #print('ids', ids)
        #print('text', texts)

        #     prpmt = []
        #     for i,j in zip(ids,texts):
        #         prpmt.append(i+'. The meme text reads :'+j + '. Classifier thinks the meme is')



        i = ii.float().to(device)
        t = ti.float().to(device)
        batch_size = i.shape[0]
        #with torch.no_grad():
        logits, m = tinymodel(i,t)
        # print(m)
        m = m.unsqueeze(dim=1)
        all_labs_clf.append(logits.argmax(dim=-1)[0].item())
        #         prmpt0, lab0 = get_tokens([tokenizer1.bos_token]*batch_size, begin=True)

        #         print(prmpt0.shape)

        gumbel_logits = F.gumbel_softmax(logits, tau=1, hard=True)
        # print(gumbel_logits)
        agmx = gumbel_logits.argmax(dim=1)

        # print(agmx, logits.argmax(dim=1))
        #         prpmt = ['model thinks the meme is ']*batch_size

        #portion1,lab1, a1 = get_tokens(prpmt)

        portion2,lab3, _ = get_tokens(['output of the classifier multimodal embedding is ']*batch_size)
        lab4 = [tokenizer1.encode('-')[:] for i in range(batch_size)]
        lab4 = torch.tensor(lab4)

        # portion3 = get_tokens(['the meme is actually'])






        kk = []
        labs_verbalized = []
        # batch_prompt = ['the model thinks the meme is ']*32
        lab2 = []
        for i in gumbel_logits:
            agmx = i.argmax()
            # print(agmx)
            # print(dix[agmx.item()])
            tokenized_ = tokenizer1.encode(dix[agmx.item()])[:]
            labs_verbalized.append(dix[agmx.item()])
            # print(tokenized_)
            lab2.append(tokenized_)
            embedding = E[tokenized_[0]]
            one_hot = i.view(-1, 1)
            # print(embedding.shape, one_hot, one_hot.shape)

            e = torch.sum(one_hot*embedding.to(device), dim=0)
            kk.append(e)
        lab2 = torch.tensor(lab2)
        prpmt = []
        for i,j,z in zip(ids,texts,labs_verbalized):
            prpmt.append(i+'. The meme text reads :'+j + '. Classifier thinks the meme is {}'.format(z))
            #all_labs_clf.append(z)
        portion1,lab1, a1 = get_tokens(prpmt)
        
        string = ['the meme is actually']
        #         if second_pass:
        #             string[0] += '{} because of'.format(llm_label)
        #             print(string)
        portion3,lab5,_ = get_tokens(string)
        inp_embed = torch.stack(kk).unsqueeze(dim=1)
        # print(inp_embed)
        # print(inp_embed.shape)
        # print(portion1.shape,inp_embed.shape,portion2.shape,m.shape,portion3.shape)

        final_embeds = torch.cat((portion1,inp_embed,portion2,m,portion3),dim=1)
        # print(final_embeds.shape)
        # print(lab1.shape, lab2.shape, lab3.shape, lab4.shape, lab5.shape)
        labs_shape = torch.cat((lab2,lab3,lab4,lab5),dim=1).shape
        remaining_mask = torch.ones(labs_shape[0], labs_shape[1]).long()
        #print(a1)
        #print(remaining_mask)

        full_mask = torch.cat((a1,remaining_mask),dim=1)
        #print('fm ', full_mask.shape)
        labs = torch.cat((lab1,lab2,lab3,lab4,lab5),dim=1)
        #print(full_mask)
        #print(0/0)
        #         print(final_embeds.shape, labs.shape)
        #         print(0/0)
        #with torch.no_grad():
        if second_pass:
            #model1.prepare_inputs_for_generation = prepare_inputs_for_generation
            #input_ids = torch.LongTensor([[model1.config.bos_token_id]]).to(device)
            #inputs_embdes = final_embeds.float().to(device)
            #op = model1.sample(input_ids, inputs_embeds = inputs_embdes, pad_token_id=model1.config.eos_token_id)
            #print(tokenizer1.batch_decode(op))
            output2 = model1(inputs_embeds = final_embeds.float(), labels = labs.long(), attention_mask=full_mask.to('cuda'))
            #print('SECOND PASS ',tokenizer1.batch_decode(output2.logits[:,-1,:].squeeze(dim=0).detach().cpu().topk(40).indices))
            second_pass_lab = tokenizer1.batch_decode(output2.logits[:,-1,:].squeeze(dim=0).detach().cpu().topk(40).indices)[0].strip()
            #print('pl', prev_lab)
            #print('sl', second_pass_lab)
            agmx_id = output2.logits.argmax(dim=-1)[0][-1].item()
            log_likelihood = output2.logits[:,-1,:]
            log_likelihood = torch.nn.functional.softmax(log_likelihood, dim=-1)[:,agmx_id][0].item()
            prev_prob = prev_prob[0].item()
            #print('current probability', log_likelihood)
            #print('previous probability', prev_prob)
            #             if (prev_lab==second_pass_lab) and (log_likelihood>=prev_prob):
            #if (prev_lab==second_pass_lab) and (abs(log_likelihood-prev_prob)<=0.05):
            #    return True
            
            if prev_lab==second_pass_lab:
                return True, log_likelihood
            
            else:
                return False, False
            
            
                                                      
        else:
            output2 = model1(inputs_embeds = final_embeds.float(), labels = labs.long(), attention_mask=full_mask.to('cuda'))
        # print(output2.loss)
        # print(0/0)
        loss_lm = output2.loss
        #print(tokenizer1.batch_decode(output2.logits.argmax(dim=-1)))

        #print(loss_lm)
        m.retain_grad()
        model1.transformer.wte.weight.retain_grad()
        #print(output2.logits.shape)
        #print(output2.logits.argmax(dim=-1)[0].shape)
        agmx_idx = output2.logits.argmax(dim=-1)[0][-1].item()
        #print(agmx_idx)
        #print(output2.logits[:,-1,:])
        log_likelihood_ = output2.logits[:,-1,:]
        log_likelihood = output2.logits[:,-1,agmx_idx]
        #print(log_likelihood)
        log_likelihood.backward()
        grad_  = m.grad.data.cpu().squeeze(1)
        #embedding_norm = model1.transformer.wte.weight.grad.data.cpu().norm(dim=-1)
        #print('en ', embedding_norm,embedding_norm.shape)
        #tokens_index = torch.mm(grad_, E.detach().cpu().T)[0]
        
        #tokens_index /= model1.transformer.wte.weight.detach().cpu().norm(dim=-1)
        #print(tokens_index)
        #ct_index = tokens_index*output2.logits[:,-1,:].squeeze(dim=0).detach().cpu() # ct_index = cumulative token index
        
        #print(tokens_index.topk(40).indices)
        #print(tokenizer1.batch_decode(embedding_norm.topk(40).indices))
        #print(tokenizer1.batch_decode(tokens_index.topk(40).indices))
        #print(tokenizer1.batch_decode(ct_index.topk(40).indices))
        #print(tokenizer1.batch_decode(output2.logits[:,-1,:].squeeze(dim=0).detach().cpu().topk(40).indices))
        #print(tokens_index.shape)
        #print(grad_.shape)
        ti,op,tt,t_id = grad_, output2.logits[:,-1,:].squeeze(dim=0).detach().cpu(), texts, tmp_id
        #print('****************************')
        #print(t_id)
        output_aware_token_idx = []
        output_aware_tokens = []
        cc = 0
        for i in op.topk(50257).indices:
            cc+=1
            k = model1.transformer.wte(i.cuda()).detach().cpu().unsqueeze(1).detach().cpu()

            #output_aware_token_idx.append(ti.mm(k))
            val = torch.nn.functional.cosine_similarity(ti.detach().cpu(),k.T)[0].item()
            
             
          
            output_aware_token_idx.append(val)
            output_aware_tokens.append(tokenizer1.decode(i))
            
            
        
        #print(output_aware_tokens)
        
        #p = list(reversed(np.argsort(output_aware_token_idx)))
        
        a,b = [],[]
        
        for i in range(len(output_aware_token_idx)):
            if output_aware_token_idx[i]>-0.1 and output_aware_token_idx[i]<0.1:
                a.append(abs(output_aware_token_idx[i]))
                b.append(output_aware_tokens[i])
            
        #print(len(a))
        
        c = np.argsort(a)
        print(len(c))
            
        d,e = [],[]
        
        for i in c:
            d.append(output_aware_token_idx[i])
            e.append(b[i])
            if len(d)==3500:
                break
            
        
        output_aware_token_idx = d
        output_aware_tokens = e
        
        
        print(len(output_aware_tokens))
        #print(0/0)
        
        #print('cc', cc)
                
           
        #         median = np.median(output_aware_token_idx)
        #         p,q = [],[]
        #         #         for i,j in zip(output_aware_token_idx, output_aware_tokens):
        #         #             if i>=median:
        #         #                 p.append(i)
        #         #                 q.append(j)

        #         for i,j in zip(output_aware_token_idx, output_aware_tokens):
        #             if i>0.5 and i<.7:
        #                 p.append(i)
        #                 q.append(j)

        #         output_aware_token_idx = p
        #         output_aware_tokens = q
        
        output_aware_tokens = list(map(lambda x: x.strip().lower(), output_aware_tokens))
        
        image = preprocess(Image.open('./data/'+t_id)).unsqueeze(0).to(device)
        text = clip.tokenize(output_aware_tokens).to(device)
        # clip_feats = []
        #text_cand = clip.tokenize(tt).to(device)



        with torch.no_grad():
            if_ = model.encode_image(image)
            #tf_ = model.encode_text(text_cand)

            #print(if_.shape, tf_.shape)
            multimodal_features = if_
            text_features = model.encode_text(text)

            clip_feats = torch.nn.functional.cosine_similarity(multimodal_features, text_features, dim=1)


        clip_feats = clip_feats.cpu().numpy().tolist()
        
        combined_feats = []
        for i,j in zip(output_aware_token_idx, clip_feats):
            #tmp = 2*i*j/(i+j)
            tmp = 1.0*j+0.00*i
            #tmp = j
            combined_feats.append(tmp)
            
        final_tokens = []
        for i in np.argsort(combined_feats)[-1:-20:-1]:
            final_tokens.append(output_aware_tokens[i])
            #print(output_aware_tokens[i])
        
        final_tokens = list(dict.fromkeys(final_tokens))
        #final_tokens = list(final_tokens)
        s = ['when', 'why', 'our']
        final_tokens = [i for i in final_tokens if (i not in s and len(i)>2)][:20]
        
        op_preserving_kt = []
        
        for i in final_tokens:
            string = '[KB] {} are'.format(i)
            op = tokenizer1.batch_decode(model1.generate(torch.tensor(tokenizer1.encode(string)).unsqueeze(0).to('cuda'), max_length=14, pad_token_id=tokenizer1.eos_token_id))
            s = ''
            for i in op[0]:
                s += i
                if i=='.':
                    break
            op_preserving_kt.append(s)
        
        #print('op preserving ', op_preserving_kt)
        #print(texts[0],labs_verbalized[0])
        #print(ids[0])
        caption = ids[0].split('[CAPTION]')[-1]
        #print(caption)
        
        kb = ''
        for i in op_preserving_kt:
            kb+=i+' '
            
        #         kb1 += '[CAPTION] '+caption + 'output of the classifier multimodal embedding is '

        #         kb2 = '. The meme text reads :'+tt[0] + '. Classifier thinks the meme is {}'.format(labs_verbalized[0]))
        
        #print('****************************')
    
        
        
        
        op_preserving_final_tokens = []
        preservation_scores = []
        llm_lab = tokenizer1.decode(output2.logits.argmax(dim=-1)[0][-1])
        #print(llm_lab)
        for i in op_preserving_kt:
            kb = i+' '
            prev_prob = torch.nn.functional.softmax(log_likelihood_, dim=-1)[:,agmx_idx]
            #print(prev_prob)
            #break
            #[:,-1,agmx_idx]
            is_op_preserved,likelihood = get_performance_test(second_pass=True, llm_label=[kb], prev_lab=llm_lab.strip(), prev_prob=prev_prob, counter=counter)
            if is_op_preserved:
                preservation_scores.append(likelihood)
                op_preserving_final_tokens.append(i)
            
                
        #print('OP PRESERVING FINAL TOKENS ',op_preserving_final_tokens)      
        #print('PRESERVATION SCORES ',preservation_scores)
        toks = []
        for idx in list(reversed(list(np.argsort(preservation_scores))))[:4]:
            #print(idx)
            tmp = op_preserving_final_tokens[idx].split('are')[0][4:].strip()
            toks.append(tmp)
        
        #print('OP PRESERVING FINAL TOKENS ',toks)
        
        string = ''
        for i in toks:
            string+=i+'\t'
        
        #print(string)
        #print(string.split('\t'))
        
        d_ids[t_id] = string
        
        all_labs.append(llm_lab)
        counter+=1
        
        
    
        
        
    return all_labs, all_labs_clf, grad_, output2.logits[:,-1,:].squeeze(dim=0).detach().cpu(), texts, tmp_id








In [ ]:
set_seed(42)
al, alc,ti,op,tt,t_id = get_performance_test()

In [ ]:
d_ids = {}
def get_performance_test(second_pass=False, llm_label='', prev_prob = 0, prev_lab = '', counter=0):
    #print(test_dataset)
    torch.use_deterministic_algorithms(True)
    set_seed(42)
    
    if second_pass:
        #print('IN SECOND PASS')
        test_dataset_ = [test_dataset_1[counter]]
        
        
    else:
        test_dataset_ = test_dataset_1
    
    tinymodel.eval()
    model1.eval()
    all_labs = []
    all_labs_clf = []
    counter = 0
    
    for ii,ti, gol, ids in tqdm(test_dataset_,disable=second_pass):

        #     print(0/0)
        #print(ids)
        tmp_id = ids
        
        #         raw_image = Image.open('./data/'+ids)
        #         if not second_pass:
        #             display(raw_image)
        ti = ti.unsqueeze(0)
        ii = ii.unsqueeze(0)
        gol = [gol]
        ids = [ids]
        # print(batch_model1, batch_model1)

        ids = list(map(lambda x: x.split('/')[1], ids))
        #         ids_ = []
        #         for i in range(len(ids)):
        #             kbs = ids[i].split('[KB]')
        #             caps = ids[i].split('[CAPTION]')[-1]
        #             try:
        #                 kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
        #             except IndexError as e:
        #                 kb = '[CAPTION] '+ caps
        #             ids_.append(kb)
        #         ids = ids_
        
        ids_ = []
        for i in range(len(ids)):
            kbs = ids[i].split('[KB]')
            #print('kbs ', kbs)
            caps = ids[i].split('[CAPTION]')[-1]
            #print(caps)
            kb = ''
            if len(kbs)>3:
                kb = '[KB] '+kbs[1]+'[KB] '+kbs[2] + '[CAPTION] '+ caps
            elif len(kbs)==3:
                #print('in len3')
                kb = '[KB] '+kbs[1]
                caps = kbs[2].split('[CAPTION]')
                kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
            elif len(kbs)==2:
                #print('in len 1')
                caps = kbs[1].split('[CAPTION]')
                kb += '[KB] '+ caps[0]+ ' [CAPTION] '+caps[1]
            elif len(kbs)==1:
                kb = kbs[0].strip()
                #print('int ', kb)



            ids_.append(kb)
        
        ids = ids_
        
        
        
        #         print(ids)
        texts = [id2text[i] for i in ids]
        ids = [kb_fb[i] for i in ids]
        if second_pass:
            ids = llm_label
        #print('ids', ids)
        #print('text', texts)

        #     prpmt = []
        #     for i,j in zip(ids,texts):
        #         prpmt.append(i+'. The meme text reads :'+j + '. Classifier thinks the meme is')



        i = ii.float().to(device)
        t = ti.float().to(device)
        batch_size = i.shape[0]
        #with torch.no_grad():
        logits, m = tinymodel(i,t)
        # print(m)
        m = m.unsqueeze(dim=1)
        all_labs_clf.append(logits.argmax(dim=-1)[0].item())
        #         prmpt0, lab0 = get_tokens([tokenizer1.bos_token]*batch_size, begin=True)

        #         print(prmpt0.shape)

        gumbel_logits = F.gumbel_softmax(logits, tau=1, hard=True)
        # print(gumbel_logits)
        agmx = gumbel_logits.argmax(dim=1)

        # print(agmx, logits.argmax(dim=1))
        #         prpmt = ['model thinks the meme is ']*batch_size

        #portion1,lab1, a1 = get_tokens(prpmt)

        portion2,lab3, _ = get_tokens(['output of the classifier multimodal embedding is ']*batch_size)
        lab4 = [tokenizer1.encode('-')[:] for i in range(batch_size)]
        lab4 = torch.tensor(lab4)

        # portion3 = get_tokens(['the meme is actually'])






        kk = []
        labs_verbalized = []
        # batch_prompt = ['the model thinks the meme is ']*32
        lab2 = []
        for i in gumbel_logits:
            agmx = i.argmax()
            # print(agmx)
            # print(dix[agmx.item()])
            tokenized_ = tokenizer1.encode(dix[agmx.item()])[:]
            labs_verbalized.append(dix[agmx.item()])
            # print(tokenized_)
            lab2.append(tokenized_)
            embedding = E[tokenized_[0]]
            one_hot = i.view(-1, 1)
            # print(embedding.shape, one_hot, one_hot.shape)

            e = torch.sum(one_hot*embedding.to(device), dim=0)
            kk.append(e)
        lab2 = torch.tensor(lab2)
        prpmt = []
        for i,j,z in zip(ids,texts,labs_verbalized):
            prpmt.append(i+'. The meme text reads :'+j + '. Classifier thinks the meme is {}'.format(z))
            #all_labs_clf.append(z)
        portion1,lab1, a1 = get_tokens(prpmt)
        
        string = ['the meme is actually']
        #         if second_pass:
        #             string[0] += '{} because of'.format(llm_label)
        #             print(string)
        portion3,lab5,_ = get_tokens(string)
        inp_embed = torch.stack(kk).unsqueeze(dim=1)
        # print(inp_embed)
        # print(inp_embed.shape)
        # print(portion1.shape,inp_embed.shape,portion2.shape,m.shape,portion3.shape)

        final_embeds = torch.cat((portion1,inp_embed,portion2,m,portion3),dim=1)
        # print(final_embeds.shape)
        # print(lab1.shape, lab2.shape, lab3.shape, lab4.shape, lab5.shape)
        labs_shape = torch.cat((lab2,lab3,lab4,lab5),dim=1).shape
        remaining_mask = torch.ones(labs_shape[0], labs_shape[1]).long()
        #print(a1)
        #print(remaining_mask)

        full_mask = torch.cat((a1,remaining_mask),dim=1)
        #print('fm ', full_mask.shape)
        labs = torch.cat((lab1,lab2,lab3,lab4,lab5),dim=1)
        #print(full_mask)
        #print(0/0)
        #         print(final_embeds.shape, labs.shape)
        #         print(0/0)
        #with torch.no_grad():
        if second_pass:
            #model1.prepare_inputs_for_generation = prepare_inputs_for_generation
            #input_ids = torch.LongTensor([[model1.config.bos_token_id]]).to(device)
            #inputs_embdes = final_embeds.float().to(device)
            #op = model1.sample(input_ids, inputs_embeds = inputs_embdes, pad_token_id=model1.config.eos_token_id)
            #print(tokenizer1.batch_decode(op))
            output2 = model1(inputs_embeds = final_embeds.float(), labels = labs.long(), attention_mask=full_mask.to('cuda'))
            #print('SECOND PASS ',tokenizer1.batch_decode(output2.logits[:,-1,:].squeeze(dim=0).detach().cpu().topk(40).indices))
            second_pass_lab = tokenizer1.batch_decode(output2.logits[:,-1,:].squeeze(dim=0).detach().cpu().topk(40).indices)[0].strip()
            #print('pl', prev_lab)
            #print('sl', second_pass_lab)
            agmx_id = output2.logits.argmax(dim=-1)[0][-1].item()
            log_likelihood = output2.logits[:,-1,:]
            log_likelihood = torch.nn.functional.softmax(log_likelihood, dim=-1)[:,agmx_id][0].item()
            prev_prob = prev_prob[0].item()
            #print('current probability', log_likelihood)
            #print('previous probability', prev_prob)
            #             if (prev_lab==second_pass_lab) and (log_likelihood>=prev_prob):
            #if (prev_lab==second_pass_lab) and (abs(log_likelihood-prev_prob)<=0.05):
            #    return True
            
            if prev_lab==second_pass_lab:
                return True, log_likelihood
            
            else:
                return False, False
            
            
                                                      
        else:
            output2 = model1(inputs_embeds = final_embeds.float(), labels = labs.long(), attention_mask=full_mask.to('cuda'))
        # print(output2.loss)
        # print(0/0)
        loss_lm = output2.loss
        #print(tokenizer1.batch_decode(output2.logits.argmax(dim=-1)))

        #print(loss_lm)
        m.retain_grad()
        model1.transformer.wte.weight.retain_grad()
        #print(output2.logits.shape)
        #print(output2.logits.argmax(dim=-1)[0].shape)
        agmx_idx = output2.logits.argmax(dim=-1)[0][-1].item()
        #print(agmx_idx)
        #print(output2.logits[:,-1,:])
        log_likelihood_ = output2.logits[:,-1,:]
        log_likelihood = output2.logits[:,-1,agmx_idx]
        #print(log_likelihood)
        log_likelihood.backward()
        grad_  = m.grad.data.cpu().squeeze(1)
        #embedding_norm = model1.transformer.wte.weight.grad.data.cpu().norm(dim=-1)
        #print('en ', embedding_norm,embedding_norm.shape)
        #tokens_index = torch.mm(grad_, E.detach().cpu().T)[0]
        
        #tokens_index /= model1.transformer.wte.weight.detach().cpu().norm(dim=-1)
        #print(tokens_index)
        #ct_index = tokens_index*output2.logits[:,-1,:].squeeze(dim=0).detach().cpu() # ct_index = cumulative token index
        
        #print(tokens_index.topk(40).indices)
        #print(tokenizer1.batch_decode(embedding_norm.topk(40).indices))
        #print(tokenizer1.batch_decode(tokens_index.topk(40).indices))
        #print(tokenizer1.batch_decode(ct_index.topk(40).indices))
        #print(tokenizer1.batch_decode(output2.logits[:,-1,:].squeeze(dim=0).detach().cpu().topk(40).indices))
        #print(tokens_index.shape)
        #print(grad_.shape)
        ti,op,tt,t_id = grad_, output2.logits[:,-1,:].squeeze(dim=0).detach().cpu(), texts, tmp_id
        #print('****************************')
        #print(t_id)
        #         output_aware_token_idx = []
        #         output_aware_tokens = []
        #         cc = 0
        #         for i in op.topk(50257).indices:
        #             cc+=1
        #             k = model1.transformer.wte(i.cuda()).detach().cpu().unsqueeze(1).detach().cpu()

        #             #output_aware_token_idx.append(ti.mm(k))
        #             val = torch.nn.functional.cosine_similarity(ti.detach().cpu(),k.T)[0].item()



        #             output_aware_token_idx.append(val)
        #             output_aware_tokens.append(tokenizer1.decode(i))

            
       
            
        
        
        image = preprocess(Image.open('./data/'+t_id)).unsqueeze(0).to(device)
        
        full = op.topk(50257).indices
        
        
        ot = []
        ot_idx = []
        
        for i in range(17):
        
            

            #for i in tqdm(full[i*0:(i+1)*1000]):
            output_aware_tokens = tokenizer1.batch_decode(full[i*3000:(i+1)*3000])
            ot.extend(output_aware_tokens)
            
            #print(output_aware_tokens[0:10])

            text = clip.tokenize(output_aware_tokens).to(device)


            with torch.no_grad():
                if_ = model.encode_image(image)

                multimodal_features = if_
                text_features = model.encode_text(text)

                clip_feats = torch.nn.functional.cosine_similarity(multimodal_features, text_features, dim=1)


            clip_feats = clip_feats.cpu().numpy().tolist()
            #print(clip_feats)
            ot_idx.extend(clip_feats)
            #print(0/0)
        
        output_aware_tokens = ot
        clip_feats = ot_idx
        print(len(output_aware_tokens), len(clip_feats))
        combined_feats = []
        
        for j in clip_feats:
            #tmp = 2*i*j/(i+j)
            tmp = 1.0*j
            #tmp = j
            combined_feats.append(tmp)
            
        final_tokens = []
        for i in np.argsort(combined_feats)[-1:-20:-1]:
            final_tokens.append(output_aware_tokens[i])
            #print(output_aware_tokens[i])
        
        final_tokens = list(dict.fromkeys(final_tokens))
        #final_tokens = list(final_tokens)
        s = ['when', 'why', 'our']
        final_tokens = [i for i in final_tokens if (i not in s and len(i)>2)][:20]
        
        op_preserving_kt = []
        
        for i in final_tokens:
            string = '[KB] {} are'.format(i)
            op = tokenizer1.batch_decode(model1.generate(torch.tensor(tokenizer1.encode(string)).unsqueeze(0).to('cuda'), max_length=14, pad_token_id=tokenizer1.eos_token_id))
            s = ''
            for i in op[0]:
                s += i
                if i=='.':
                    break
            op_preserving_kt.append(s)
        
        #print('op preserving ', op_preserving_kt)
        #print(texts[0],labs_verbalized[0])
        #print(ids[0])
        caption = ids[0].split('[CAPTION]')[-1]
        #print(caption)
        
        kb = ''
        for i in op_preserving_kt:
            kb+=i+' '
            
        #         kb1 += '[CAPTION] '+caption + 'output of the classifier multimodal embedding is '

        #         kb2 = '. The meme text reads :'+tt[0] + '. Classifier thinks the meme is {}'.format(labs_verbalized[0]))
        
        #print('****************************')
    
        
        
        
        op_preserving_final_tokens = []
        preservation_scores = []
        llm_lab = tokenizer1.decode(output2.logits.argmax(dim=-1)[0][-1])
        #print(llm_lab)
        for i in op_preserving_kt:
            kb = i+' '
            prev_prob = torch.nn.functional.softmax(log_likelihood_, dim=-1)[:,agmx_idx]
            #print(prev_prob)
            #break
            #[:,-1,agmx_idx]
            is_op_preserved,likelihood = get_performance_test(second_pass=True, llm_label=[kb], prev_lab=llm_lab.strip(), prev_prob=prev_prob, counter=counter)
            if is_op_preserved:
                preservation_scores.append(likelihood)
                op_preserving_final_tokens.append(i)
            
                
        #print('OP PRESERVING FINAL TOKENS ',op_preserving_final_tokens)      
        #print('PRESERVATION SCORES ',preservation_scores)
        toks = []
        for idx in list(reversed(list(np.argsort(preservation_scores))))[:4]:
            #print(idx)
            tmp = op_preserving_final_tokens[idx].split('are')[0][4:].strip()
            toks.append(tmp)
        
        #print('OP PRESERVING FINAL TOKENS ',toks)
        
        string = ''
        for i in toks:
            string+=i+'\t'
        
        #print(string)
        #print(string.split('\t'))
        
        d_ids[t_id] = string
        
        all_labs.append(llm_lab)
        counter+=1
        
        
    
        
        
    return all_labs, all_labs_clf, grad_, output2.logits[:,-1,:].squeeze(dim=0).detach().cpu(), texts, tmp_id








In [ ]:
print('ok')

In [ ]:
set_seed(42)
al, alc,ti,op,tt,t_id = get_performance_test()

In [ ]:
set_seed(42)
al, alc,ti,op,tt,t_id = get_performance_test()

In [ ]:
d_ids

In [ ]:
d_ids

In [ ]:

import pickle

# open a file, where you stored the pickled data
file = open('/kaggle/input/extras/iclr_2500_-0.1_0.1_3500ex_on_on.pkl', 'rb')

# dump information to that file
data = pickle.load(file)

# close the file
file.close()

In [ ]:
data[0]

In [ ]:
raw_image = Image.open('./data/'+'img/39862.png')
display(raw_image)

In [ ]:
d_ids

In [ ]:
d_ids

In [ ]:
raw_image = Image.open('./data/'+'img/63784.png')
display(raw_image)

In [ ]:
al

In [ ]:
all_labs = []
for i in al:
    if i.strip()=='normal':
        all_labs.append(0)
    else:
        all_labs.append(1)

In [ ]:
all_labs

In [ ]:
al_dict = {}
for i,j in zip(d_ids,range(100)):
    al_dict[i] = all_labs[j]

In [ ]:
d_ids

In [ ]:
al_dict

In [ ]:
d_ids

In [ ]:
d_ids

In [ ]:
import pickle

list_of_dicts = [d_ids,all_labs]

with open('iclr_3500_ur_on_on.pkl', 'wb') as f:
    pickle.dump(list_of_dicts, f)

In [ ]:
with open('iclr_ur_ur_on_on.pkl', 'rb') as f:
    lod = pickle.load(f)

In [ ]:
lod

In [ ]:
import torch

class TinyModel_(torch.nn.Module):

    def __init__(self, mdl, mdl_rand, rand=False):
        super(TinyModel_, self).__init__()
       
        self.linear11 = nn.Sequential(nn.Linear(25, 512), nn.ReLU(), nn.Linear(512, 768))
        
        self.linear1 = mdl_rand
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(768, 128)
        self.linear3 = torch.nn.Linear(128, 2)
        self.rand = rand
        self.softmax = torch.nn.Softmax()
        self.proj = torch.nn.Linear(512,768)
        # self.proj = torch.nn.Linear(512,5120)

    def forward(self, x, y, z,p=0,train=True, no_exp=False, only_exp=False, all_exp=False):
        joint_tensor = torch.cat((x, y), dim=1)
        z = self.activation(z)
        if self.rand:
            m_ = self.linear1(joint_tensor)
        else:
            x = x.unsqueeze(1)
            y = y.unsqueeze(1)
            m_ = self.linear1(x, y).squeeze(dim=1)
            
        
        m_ = self.activation(m_)
        if train:
            #print(p)
            a1,a2,a3 = [],[],[]
            for i in p:
                if i==0:
                    a1.append(1)
                    a2.append(0)
                    #a3.append(0)
                elif i==1:
                    a1.append(0)
                    a2.append(1)
                    #a3.append(0)
                #                 elif i==2:
                #                     a1.append(0)
                #                     a2.append(0)
                #                     a3.append(1)
            #print('a1',a1)
            #print('proj', self.proj(m_.to(dtype=torch.float32)))
            
            a1 = torch.tensor(a1).view(-1,1).to(device)
            a2 = torch.tensor(a2).view(-1,1).to(device)
            #a3 = torch.tensor(a3).view(-1,1).to(device)
            #print(a1*self.proj(m_.to(dtype=torch.float32)))
            #m = a1*self.proj(m_.to(dtype=torch.float32)) + a2*self.linear11(z.to(dtype=torch.float32)) + a3*(self.proj(m_.to(dtype=torch.float32))+self.linear11(z.to(dtype=torch.float32)))
            #m = a1*self.proj(m_.to(dtype=torch.float32)) + a2*self.linear11(z.to(dtype=torch.float32)) 
            m = (self.proj(m_.to(dtype=torch.float32))*self.linear11(z.to(dtype=torch.float32)))
        if not train:
            if no_exp:
                #print('a')
                m = self.proj(m_.to(dtype=torch.float32))
                
            if only_exp:
                #print('b')
                m = self.linear11(z.to(dtype=torch.float32))

            if all_exp:
                #print('c')
                m = (self.proj(m_.to(dtype=torch.float32))*self.linear11(z.to(dtype=torch.float32)))

        m = self.linear3(self.linear2(m))
        # m = self.softmax(m)
        return m, m_



In [ ]:
!pip install gdown

In [ ]:
# !gdown https://drive.google.com/u/0/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM

In [ ]:
import gensim.downloader as api

glove_vectors = api.load('glove-twitter-25')

In [ ]:
print('ok')

In [ ]:
a,b = 0,0
p = []
c
    p.append(k
    if k==0:
        a+=1
    elif k==1:
        b+=1

In [ ]:
import random

In [ ]:
va,b = 0,0

random.seed(123)
k = random.randint(0,1)
k

In [ ]:
p

In [ ]:
len(dataloader_test)

In [ ]:
import os
os.listdir('/kaggle/input/tensors-models')

In [ ]:
set_seed(42)
rnd = np.random.randn(25)

In [ ]:
rnd

In [ ]:
import torch
#os.environ['CUBLAS_WORKSPACE_CONFIG']=:4096:8
#/kaggle/input/extras/iclr_2500_-0.1_0.1_on_on.pkl

for i in os.listdir('/kaggle/input/tensors-models'):
    print(i)
    with open('/kaggle/input/tensors-models/'+i, 'rb') as f:
        lod = pickle.load(f)
    test_dataset = torch.utils.data.Subset(test_dataset_1, [i for i in range(71,100)])
    d_ids = lod[0]
    all_labs = lod[1]
    torch.use_deterministic_algorithms(True)
    mean = []
    std = []
    for i in range(10):
        set_seed(i*13)
        p = []
        for i in range(100):
            p.append(random.randint(0,1))
        import numpy as np

        mdl_rand = fusion(512,512,True,256,512,0.1).to(device)
        tinymodel_ = TinyModel_(mdl,mdl_rand,rand=False).to(device)


        optimizer = optim.Adam(tinymodel_.parameters(), lr=0.0005)

        dix = {0:'normal', 1:'offensive'}

        tinymodel_.train()

        # Training loop
        num_epochs = 30
        def transpose(x):
            return x.transpose(-2, -1)

        for epoch in tqdm(range(num_epochs)):
            #     set_seed(42)
            l=0
            cnt = 0
            l_lm = 0
            l_clf = 0
            counter = 0
            for ii,ti, gol, ids in dataloader_test:
                # print(batch_model1, batch_model1)

                #         print(ii.shape)
                #         print(ti.shape)
                #         print(gol)
                #         print(ids)

                explanations = []
                gold_label = []
                for id_ in ids:
                    gold_label.append(al_dict[id_])

                gold_label = torch.tensor(gold_label)








                #ids = list(map(lambda x: x.split('/')[1], ids))
                #         print(ids)


                we = []

                for id_ in ids: 
                    tokens = [i for i in d_ids[id_].split('\t') if i!='']
                    #print(tokens)

                    word_embeddings = []

                    for token in tokens:
                        if token in glove_vectors.index_to_key:
                            word_embeddings.append(glove_vectors[token])

                    # Calculate the average embedding
                    if word_embeddings:
                        avg_embedding = np.mean(word_embeddings, axis=0)
                    else:
                        # Handle the case where none of the words were found in the vocabulary
                        avg_embedding = rnd  # You can use a zero vector or other defaults

                    we.append(avg_embedding)

                we = np.asarray(we)


                #print(we.shape)





                i = ii.float().to(device)
                t = ti.float().to(device)
                #print(counter)
                batch_size = i.shape[0]
                indices = p[counter*batch_size:(counter+1)*batch_size]
                #print(indices)

                optimizer.zero_grad()

                logits, m = tinymodel_(i,t, torch.tensor(we).to(device),p=indices, train=True)

                #print(0/0)



                #print(logits)

                gol = torch.tensor(all_labs[counter*batch_size:(counter+1)*batch_size])
                loss_clf = torch.nn.functional.cross_entropy(logits, gold_label.to(device), reduction='mean')



                loss_clf.backward()
                optimizer.step()




                l_clf+=loss_clf.detach().item()

                cnt+=1
                counter+=1
            #if epoch%5==0:
            #print(f"Epoch [{epoch+1}/{num_epochs}], Loss clf: {l_clf/cnt}")



        print("Training complete!")
        leakage = []
        non_leakage = []
        global_dict = {}
        tinymodel_.eval()
        counter=0
        for ii,ti, gol, ids in tqdm(test_dataset):
            we = []
            ii = ii.unsqueeze(dim=0)
            ti = ti.unsqueeze(dim=0)
            gol = [al_dict[ids]]
            ids = [ids]

            for id_ in ids: 
                tokens = [i for i in d_ids[id_].split('\t') if i!='']
                #print(tokens)

                word_embeddings = []

                for token in tokens:
                    if token in glove_vectors.index_to_key:
                        word_embeddings.append(glove_vectors[token])

                # Calculate the average embedding
                if word_embeddings:
                    avg_embedding = np.mean(word_embeddings, axis=0)
                else:
                    # Handle the case where none of the words were found in the vocabulary
                    avg_embedding = np.zeros(25)  # You can use a zero vector or other defaults

                we.append(avg_embedding)

            we = np.asarray(we)


            #print(we.shape)





            i = ii.float().to(device)
            t = ti.float().to(device)


            batch_size = i.shape[0]
            optimizer.zero_grad()

            logits, m = tinymodel_(i,t, torch.tensor(we).to(device), train=False, all_exp=True)
            logits_no_exp,_ = tinymodel_(i,t, torch.tensor(we).to(device), train=False, no_exp=True)
            logits_only_exp,_ = tinymodel_(i,t, torch.tensor(we).to(device), train=False,only_exp=True)



            #print(gol)
            #print(logits_only_exp.argmax(dim=-1).item(), logits.argmax(dim=-1).item(), logits_no_exp.argmax(dim=-1).item())

            if gol[0]==logits_only_exp.argmax(dim=-1).item():
                leakage.append(ids[0])
            elif gol[0]!=logits_only_exp.argmax(dim=-1).item():
                non_leakage.append(ids[0]) 

            l = int(gol[0]==logits.argmax(dim=-1).item()) - int(gol[0]==logits_no_exp.argmax(dim=-1).item())

            global_dict[ids[0]] = l
            counter+=1


            #print(0/0)
        l1,l2 = 0,0
        for i in leakage:
            l1+=global_dict[i]
        l1 = l1/len(leakage)

        for i in non_leakage:
            l2+=global_dict[i]
        l2 = l2/len(non_leakage)

        print(l1,l2,len(leakage),len(non_leakage),0.5*l1+0.5*l2)
        mean.append(0.5*l1+0.5*l2)
    
    print(np.mean(mean), np.std(mean))



    

In [ ]:
import torch
#os.environ['CUBLAS_WORKSPACE_CONFIG']=:4096:8
#/kaggle/input/extras/iclr_2500_-0.1_0.1_on_on.pkl

for i in os.listdir('/kaggle/input/extras'):
    print(i)
    with open('/kaggle/input/extras/'+i, 'rb') as f:
        lod = pickle.load(f)
    test_dataset = torch.utils.data.Subset(test_dataset_1, [i for i in range(71,100)])
    d_ids = lod[0]
    all_labs = lod[1]
    torch.use_deterministic_algorithms(True)
    mean = []
    std = []
    for i in range(10):
        set_seed(i*13)
        p = []
        for i in range(100):
            p.append(random.randint(0,1))
        import numpy as np

        mdl_rand = fusion(512,512,True,256,512,0.1).to(device)
        tinymodel_ = TinyModel_(mdl,mdl_rand,rand=False).to(device)


        optimizer = optim.Adam(tinymodel_.parameters(), lr=0.0005)

        dix = {0:'normal', 1:'offensive'}

        tinymodel_.train()

        # Training loop
        num_epochs = 30
        def transpose(x):
            return x.transpose(-2, -1)

        for epoch in tqdm(range(num_epochs)):
            #     set_seed(42)
            l=0
            cnt = 0
            l_lm = 0
            l_clf = 0
            counter = 0
            for ii,ti, gol, ids in dataloader_test:
                # print(batch_model1, batch_model1)

                #         print(ii.shape)
                #         print(ti.shape)
                #         print(gol)
                #         print(ids)

                explanations = []
                gold_label = []
                for id_ in ids:
                    gold_label.append(al_dict[id_])

                gold_label = torch.tensor(gold_label)








                #ids = list(map(lambda x: x.split('/')[1], ids))
                #         print(ids)


                we = []

                for id_ in ids: 
                    tokens = [i for i in d_ids[id_].split('\t') if i!='']
                    #print(tokens)

                    word_embeddings = []

                    for token in tokens:
                        if token in glove_vectors.index_to_key:
                            word_embeddings.append(glove_vectors[token])

                    # Calculate the average embedding
                    if word_embeddings:
                        avg_embedding = np.mean(word_embeddings, axis=0)
                    else:
                        # Handle the case where none of the words were found in the vocabulary
                        avg_embedding = rnd  # You can use a zero vector or other defaults

                    we.append(avg_embedding)

                we = np.asarray(we)


                #print(we.shape)





                i = ii.float().to(device)
                t = ti.float().to(device)
                #print(counter)
                batch_size = i.shape[0]
                indices = p[counter*batch_size:(counter+1)*batch_size]
                #print(indices)

                optimizer.zero_grad()

                logits, m = tinymodel_(i,t, torch.tensor(we).to(device),p=indices, train=True)

                #print(0/0)



                #print(logits)

                gol = torch.tensor(all_labs[counter*batch_size:(counter+1)*batch_size])
                loss_clf = torch.nn.functional.cross_entropy(logits, gold_label.to(device), reduction='mean')



                loss_clf.backward()
                optimizer.step()




                l_clf+=loss_clf.detach().item()

                cnt+=1
                counter+=1
            #if epoch%5==0:
            #print(f"Epoch [{epoch+1}/{num_epochs}], Loss clf: {l_clf/cnt}")



        print("Training complete!")
        leakage = []
        non_leakage = []
        global_dict = {}
        tinymodel_.eval()
        counter=0
        for ii,ti, gol, ids in tqdm(test_dataset):
            we = []
            ii = ii.unsqueeze(dim=0)
            ti = ti.unsqueeze(dim=0)
            gol = [al_dict[ids]]
            ids = [ids]

            for id_ in ids: 
                tokens = [i for i in d_ids[id_].split('\t') if i!='']
                #print(tokens)

                word_embeddings = []

                for token in tokens:
                    if token in glove_vectors.index_to_key:
                        word_embeddings.append(glove_vectors[token])

                # Calculate the average embedding
                if word_embeddings:
                    avg_embedding = np.mean(word_embeddings, axis=0)
                else:
                    # Handle the case where none of the words were found in the vocabulary
                    avg_embedding = np.zeros(25)  # You can use a zero vector or other defaults

                we.append(avg_embedding)

            we = np.asarray(we)


            #print(we.shape)





            i = ii.float().to(device)
            t = ti.float().to(device)


            batch_size = i.shape[0]
            optimizer.zero_grad()

            logits, m = tinymodel_(i,t, torch.tensor(we).to(device), train=False, all_exp=True)
            logits_no_exp,_ = tinymodel_(i,t, torch.tensor(we).to(device), train=False, no_exp=True)
            logits_only_exp,_ = tinymodel_(i,t, torch.tensor(we).to(device), train=False,only_exp=True)



            #print(gol)
            #print(logits_only_exp.argmax(dim=-1).item(), logits.argmax(dim=-1).item(), logits_no_exp.argmax(dim=-1).item())

            if gol[0]==logits_only_exp.argmax(dim=-1).item():
                leakage.append(ids[0])
            elif gol[0]!=logits_only_exp.argmax(dim=-1).item():
                non_leakage.append(ids[0]) 

            l = int(gol[0]==logits.argmax(dim=-1).item()) - int(gol[0]==logits_no_exp.argmax(dim=-1).item())

            global_dict[ids[0]] = l
            counter+=1


            #print(0/0)
        l1,l2 = 0,0
        for i in leakage:
            l1+=global_dict[i]
        l1 = l1/len(leakage)

        for i in non_leakage:
            l2+=global_dict[i]
        l2 = l2/len(non_leakage)

        print(l1,l2,len(leakage),len(non_leakage),0.5*l1+0.5*l2)
        mean.append(0.5*l1+0.5*l2)
    
    print(np.mean(mean), np.std(mean))



    

In [ ]:
for i in 'republican\tdemocratic\tpolitician\tliberal\t'.split('\t')[:-1]:
    print(glove_vectors[i])

In [ ]:
for i in test_dataset_1:
    print(i[2])

In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
set_seed(42)
rnd = np.random.randn(25)
def get_repr(i):
    tokens = i.split('\t')[:-1] if i!='' else ['']
    #print(tokens)

    word_embeddings = []

    for token in tokens:
        if token in glove_vectors.index_to_key:
            word_embeddings.append(glove_vectors[token])
    print(word_embeddings)
    # Calculate the average embedding
    if word_embeddings:
        avg_embedding = np.mean(word_embeddings, axis=0)
    else:
        avg_embedding = rnd
        
    return avg_embedding
    
for i in os.listdir('/kaggle/input/full-ops'):
    print(i)
    with open('/kaggle/input/full-ops/'+i, 'rb') as f:
        lod = pickle.load(f)
        y = lod[1]
        k = []
        p = []
        for i in test_dataset:
            k.append(get_repr(lod[0][i[3]]))
            a,b= i[0],i[1]
            p.append(np.concatenate([a,b],axis=-1))

    k = np.asarray(k)
    p = np.asarray(p)
    
    X = np.concatenate([k,p],axis=-1) # inp w/ clip representation w/ exp
    X_ = p # inp w/ clip representation w/o exp
    X__ = k # inp w/o clip representation w/ exp
    
    from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
    from sklearn import svm
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    #clf = svm.SVC(kernel='poly', C=2, random_state=42)
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
    kf = KFold(n_splits=5)
    y = np.array(y)
    vals = []
    for train, test in kf.split(X):
        
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
        X_train_, X_test_, y_train, y_test = X_[train], X_[test], y[train], y[test]
        X_train__, X_test__, y_train, y_test = X__[train], X__[test], y[train], y[test]
        
        
        clf1 = make_pipeline(StandardScaler(),GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0))
        clf1.fit(X_train, y_train)
        y_pred = clf1.predict(X_test)
        
        clf2 = make_pipeline(StandardScaler(),GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0))
        clf2.fit(X_train_, y_train)
        y_pred_ = clf2.predict(X_test_)
        
        clf3 = make_pipeline(StandardScaler(), GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0))
        clf3.fit(X_train__, y_train)
        y_pred__ = clf3.predict(X_test__)
        
        l, nl = 0,0
        
        for i ,j in zip(y_pred__,y_test):
            if i==j:
                l+=1
            else:
                nl+=1
        a,b=0,0
        for i,j,k,x in zip(y_pred,y_pred_,y_test,y_pred__):
            if k==x:
                a += int(i==k) - int(j==k)
            elif k!=x:
                b += int(i==k) - int(j==k)
        
        print(.5*(a/l)+.5*(b/nl))
        vals.append(.5*(a/l)+.5*(b/nl))
    print('mean ', np.mean(vals), 'std ', np.std(vals))
    
            
        
        
        
        
        
        
        
        
        
        
        
        
    #     y_pred = cross_val_predict(clf, X, y, cv=3)
    #     y_pred_ = cross_val_predict(clf, X_, y, cv=3)
    #     y_pred__ = cross_val_predict(clf, X__, y, cv=3)

    #     print(y_pred)
    #     print(y_pred.shape)

    #     print(y_pred_)
    #     print(y_pred_.shape)

    #     print(0/0)
    
    
    

        

In [ ]:
import pickle

In [ ]:
files = ['/kaggle/input/full-ops/iclr_3500_-0.10.1_on_on.pkl', '/kaggle/input/full-ops/iclr_ur_0.01_ball_on_on.pkl', '/kaggle/input/full-ops/iclr_baseline_ig.pkl', '/kaggle/input/full-ops/iclr_ur_ur_on_on.pkl']

In [ ]:
import random
random.seed(420)
k = random.sample([i for i in range(1700)], 900)

In [ ]:

kk = {}
for file in files:
    print(file)
    ids_ = []
    kk[file] = []
    with open(file, 'rb') as f:
        lod = pickle.load(f)
        dic = lod[0]
        #         print(dic)
        #         break
        counter = 0
        for i,lo in zip(dic,lod[1]):
            
            ids_.append(i)
            kk[file].append(dic[i])

            pred = lo
            #act = id2text[i[4:]]
            #print(i)
            #break
            if str(i)=='img/91768.png':
                print(i, dic[i])
                break
                    
            counter+=1
            
         
    print('**')
            

In [ ]:

kk = {}
for file in files:
    print(file)
    ids_ = []
    kk[file] = []
    with open(file, 'rb') as f:
        lod = pickle.load(f)
        dic = lod[0]
        counter = 0
        for i,lo in zip(dic,lod[1]):
            if counter in k:
                ids_.append(i)
                kk[file].append(dic[i])
                
                pred = lo
                act = id2text[i[4:]]
                
                if pred==0 and act==0:
                    print(i, dic[i])
                    print(pred,act) #1,0
                    
            counter+=1
            if counter==700:
                break
         
    print('**')
            

In [ ]:
k = ['img/52978.png', 'img/28197.png']
for i,j in zip(lod[0],lod[1]):
    if i in k:
        print(i,j)

In [ ]:
import pandas as pd
df = pd.DataFrame(kk)

In [ ]:
df.head(10)

In [ ]:
df.to_csv('human_eval.csv', sep=',')

In [ ]:
import os
os.listdir('/kaggle/working/')

In [ ]:
df = pd.read_csv('/kaggle/working/human_eval.csv')

In [ ]:
df.head(10)

In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import *
from scipy.spatial import distance
import pickle
set_seed(42)
rnd = np.random.randn(25)

#loc = ['iclr_baseline_ig.pkl', 'iclr_baseline_ixg.pkl', 'iclr_baseline_ks.pkl', 'iclr_baseline_saliency.pkl', 'iclr_500_ur_on_on.pkl']

#loc = ['iclr_2500_0.1_on_off.pkl', 'iclr_500_0.1_on_off.pkl', 'iclr_3500_ball_0.1_on_off.pkl', 'iclr_1500_0.1_on_off.pkl', 'iclr_2500_-0.10.1_on_on.pkl', 'iclr_500_ball_0.1_on_on.pkl', 'iclr_3500_ball_-0.10.1_on_on.pkl', 'iclr_1500_ball_0.1_on_on.pkl']
loc = ['iclr_3500_ball_0.1_on_off.pkl',  'iclr_3500_-0.10.1_on_on.pkl']

def get_repr(i,CNT):
    tokens = i.split('\t')[:-1] if i!='' else ['']
    #print(tokens)

    word_embeddings = []
    intra_dist = []
    for token in tokens:
        if token in glove_vectors.index_to_key:
            word_embeddings.append(glove_vectors[token])
    
    # Calculate the average embedding
    if word_embeddings:
        avg_embedding = np.mean(word_embeddings, axis=0)
    else:
        avg_embedding = rnd
        CNT+=1
    
    
    for we in word_embeddings:
        #dist = np.linalg.norm(we-avg_embedding)
        dist = distance.sqeuclidean(we, avg_embedding)
        intra_dist.append(dist)
    
    intra_dist = np.mean(intra_dist)
    
    return avg_embedding, CNT, intra_dist
    
for i in os.listdir('/kaggle/input/full-ops'):
    print(i)
    if i not in loc:
        continue
    CNT=0
    with open('/kaggle/input/full-ops/'+i, 'rb') as f:
        lod = pickle.load(f)
        y = lod[1]
        k = []
        p = []
        ID = []
        ID1 = []
        for i in test_dataset:
            embed, CNT, intra_dist = get_repr(lod[0][i[3]], CNT)
            k.append(embed)
            ID.append(intra_dist)
            a,b= i[0],i[1]
            p.append(np.concatenate([a,b],axis=-1))
    
    print(CNT)
    ID = np.nan_to_num(ID)
    ID_mean = np.mean(np.asarray(k), axis=0)
    
    for i in k:
        #dist = np.linalg.norm(i-ID_mean)
        dist = distance.sqeuclidean(i, ID_mean)
        ID1.append(dist)
        
    
    ID1 = np.nan_to_num(ID1)
    k = np.asarray(k)
    p = np.asarray(p)
    
    X = np.concatenate([k,p],axis=-1) # inp w/ clip representation w/ exp -> w/ both
    X_ = p # inp w/ clip representation w/o exp -> w/ only input
    X__ = k # inp w/o clip representation w/ exp -> w/ only exp
    
    from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
    from sklearn import svm
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    #clf = svm.SVC(kernel='poly', C=2, random_state=42)
    
    kf = KFold(n_splits=4)
    y = np.array(y)
    vals = []
    vals_lus = []
    k1,k2,k3 = [],[],[]
    C,S = [],[]
    for train, test in kf.split(X):
        
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
        X_train_, X_test_, y_train, y_test = X_[train], X_[test], y[train], y[test]
        X_train__, X_test__, y_train, y_test = X__[train], X__[test], y[train], y[test]
        
        
        clf1 = make_pipeline(StandardScaler(),svm.SVC(probability=True, kernel='rbf', C=2, random_state=42))
        clf1.fit(X_train, y_train)
        y_pred = clf1.predict(X_test)
        
        clf2 = make_pipeline(StandardScaler(),svm.SVC(probability=True,kernel='rbf', C=2, random_state=42))
        clf2.fit(X_train_, y_train)
        y_pred_ = clf2.predict(X_test_)
        
        clf3 = make_pipeline(StandardScaler(), svm.SVC(probability=True,kernel='rbf', C=2, random_state=42))
        clf3.fit(X_train__, y_train)
        y_pred__ = clf3.predict(X_test__)
        
        
        y_pred_proba_w_both = clf1.predict_proba(X_test)
        y_pred_proba_w_inp = clf2.predict_proba(X_test_)
        y_pred_proba_w_exp = clf3.predict_proba(X_test__)
        
        
        
        comprehensiveness, sufficiency = 0,0
        
        counter = 0
        for pred_proba_w_both, pred_proba_w_inp, pred_proba_w_exp, pred_class in zip(y_pred_proba_w_both,y_pred_proba_w_inp,y_pred_proba_w_exp,y_pred):
            comprehensiveness += (pred_proba_w_both[pred_class] - pred_proba_w_inp[pred_class])
            sufficiency += (pred_proba_w_both[pred_class] - pred_proba_w_exp[pred_class])
            counter+=1
        
        
        comprehensiveness = comprehensiveness / counter
        sufficiency = sufficiency / counter
            
        C.append(comprehensiveness)
        S.append(sufficiency)
            
            
            
       
        
        l, nl = 0,0
        us = 0
        for i ,j in zip(y_pred__,y_test):
            if i==j:
                l+=1
            else:
                nl+=1
        a,b=0,0
        for i,j,k,x in zip(y_pred,y_pred_,y_test,y_pred__):
            us+=int(i==k) - int(j==k)
            if k==x:
                a += int(i==k) - int(j==k)
            elif k!=x:
                b += int(i==k) - int(j==k)
        
        #print('LAS ',.5*(a/l)+.5*(b/nl))
        #print('Leakage Unadjusted Simulatability (LUS)', (us/(l+nl)))
        vals_lus.append(us/(l+nl))
        #print('Comprehensiveness ', comprehensiveness)
        #print('Sufficiency ', sufficiency)
        #print('f1 between prediction of proposed model and SVM w/ both inp and exp {}'.format(f1_score(y_test, y_pred, average='macro')))
        #print('f1 between prediction of proposed model and SVM w/ only inp {}'.format(f1_score(y_test, y_pred_, average='macro')))
        #print('f1 between prediction of proposed model and SVM w/ only exp {}'.format(f1_score(y_test, y_pred__, average='macro')))
        vals.append(.5*(a/l)+.5*(b/nl))
        k1.append(f1_score(y_test, y_pred, average='macro'))
        k2.append(f1_score(y_test, y_pred_, average='macro'))
        k3.append(f1_score(y_test, y_pred__, average='macro'))
        
        
    print('Intra mean ', np.mean(ID), 'Intra std ', np.std(ID))
    print('Inter mean ', np.mean(ID1), 'Inter std ', np.std(ID1))
    print('LAS mean ', np.mean(vals), 'LAS std ', np.std(vals))
    print('LUS mean ', np.mean(vals_lus), 'LUS std ', np.std(vals_lus))
    print('Comprehensiveness mean ', np.mean(C), 'Comprehensiveness std ', np.std(C))
    print('Sufficiency mean ', np.mean(S), 'Sufficiency std ', np.std(S))
    print('Avg. f1 between prediction of proposed model and SVM w/ both inp and exp {}'.format(np.mean(k1)))
    print('Avg. f1 between prediction of proposed model and SVM w/ only inp {}'.format(np.mean(k2)))
    print('Avg. f1 between prediction of proposed model and SVM w/ only exp {}'.format(np.mean(k3)))
    
            
        
        
        
        
        
        
        
        
        
        
        
        
        
    #     y_pred = cross_val_predict(clf, X, y, cv=3)
    #     y_pred_ = cross_val_predict(clf, X_, y, cv=3)
    #     y_pred__ = cross_val_predict(clf, X__, y, cv=3)

    #     print(y_pred)
    #     print(y_pred.shape)

    #     print(y_pred_)
    #     print(y_pred_.shape)

    #     print(0/0)
    
    
    

        

In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import *
from scipy.spatial import distance
import pickle
set_seed(42)
rnd = np.random.randn(25)

def get_repr(i,CNT):
    tokens = i.split('\t')[:-1] if i!='' else ['']
    #print(tokens)

    word_embeddings = []
    intra_dist = []
    for token in tokens:
        if token in glove_vectors.index_to_key:
            word_embeddings.append(glove_vectors[token])
    
    # Calculate the average embedding
    if word_embeddings:
        avg_embedding = np.mean(word_embeddings, axis=0)
    else:
        avg_embedding = rnd
        CNT+=1
    
    
    for we in word_embeddings:
        #dist = np.linalg.norm(we-avg_embedding)
        dist = distance.sqeuclidean(we, avg_embedding)
        intra_dist.append(dist)
    
    intra_dist = np.mean(intra_dist)
    
    return avg_embedding, CNT, intra_dist
    
for i in os.listdir('/kaggle/input/full-ops'):
    print(i)
    CNT=0
    with open('/kaggle/input/full-ops/'+i, 'rb') as f:
        lod = pickle.load(f)
        y = lod[1]
        k = []
        p = []
        ID = []
        ID1 = []
        for i in test_dataset:
            embed, CNT, intra_dist = get_repr(lod[0][i[3]], CNT)
            k.append(embed)
            ID.append(intra_dist)
            a,b= i[0],i[1]
            p.append(np.concatenate([a,b],axis=-1))
    
    print(CNT)
    ID = np.nan_to_num(ID)
    ID_mean = np.mean(np.asarray(k), axis=0)
    
    for i in k:
        #dist = np.linalg.norm(i-ID_mean)
        dist = distance.sqeuclidean(i, ID_mean)
        ID1.append(dist)
        
    
    ID1 = np.nan_to_num(ID1)
    k = np.asarray(k)
    p = np.asarray(p)
    
    X = np.concatenate([k,p],axis=-1) # inp w/ clip representation w/ exp -> w/ both
    X_ = p # inp w/ clip representation w/o exp -> w/ only input
    X__ = k # inp w/o clip representation w/ exp -> w/ only exp
    
    from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
    from sklearn import svm
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    #clf = svm.SVC(kernel='poly', C=2, random_state=42)
    
    kf = KFold(n_splits=4)
    y = np.array(y)
    vals = []
    vals_lus = []
    k1,k2,k3 = [],[],[]
    C,S = [],[]
    for train, test in kf.split(X):
        
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
        X_train_, X_test_, y_train, y_test = X_[train], X_[test], y[train], y[test]
        X_train__, X_test__, y_train, y_test = X__[train], X__[test], y[train], y[test]
        
        
        clf1 = make_pipeline(StandardScaler(),svm.SVC(probability=True, kernel='rbf', C=2, random_state=42))
        clf1.fit(X_train, y_train)
        y_pred = clf1.predict(X_test)
        
        clf2 = make_pipeline(StandardScaler(),svm.SVC(probability=True,kernel='rbf', C=2, random_state=42))
        clf2.fit(X_train_, y_train)
        y_pred_ = clf2.predict(X_test_)
        
        clf3 = make_pipeline(StandardScaler(), svm.SVC(probability=True,kernel='rbf', C=2, random_state=42))
        clf3.fit(X_train__, y_train)
        y_pred__ = clf3.predict(X_test__)
        
        
        y_pred_proba_w_both = clf1.predict_proba(X_test)
        y_pred_proba_w_inp = clf2.predict_proba(X_test_)
        y_pred_proba_w_exp = clf3.predict_proba(X_test__)
        
        
        
        comprehensiveness, sufficiency = 0,0
        
        counter = 0
        for pred_proba_w_both, pred_proba_w_inp, pred_proba_w_exp, pred_class in zip(y_pred_proba_w_both,y_pred_proba_w_inp,y_pred_proba_w_exp,y_pred):
            comprehensiveness += (pred_proba_w_both[pred_class] - pred_proba_w_inp[pred_class])
            sufficiency += (pred_proba_w_both[pred_class] - pred_proba_w_exp[pred_class])
            counter+=1
        
        
        comprehensiveness = comprehensiveness / counter
        sufficiency = sufficiency / counter
            
        C.append(comprehensiveness)
        S.append(sufficiency)
            
            
            
       
        
        l, nl = 0,0
        us = 0
        for i ,j in zip(y_pred__,y_test):
            if i==j:
                l+=1
            else:
                nl+=1
        a,b=0,0
        for i,j,k,x in zip(y_pred,y_pred_,y_test,y_pred__):
            us+=int(i==k) - int(j==k)
            if k==x:
                a += int(i==k) - int(j==k)
            elif k!=x:
                b += int(i==k) - int(j==k)
        
        #         print('LAS ',.5*(a/l)+.5*(b/nl))
        #         print('Leakage Unadjusted Simulatability (LUS)', (us/(l+nl)))
        vals_lus.append(us/(l+nl))
        #         print('Comprehensiveness ', comprehensiveness)
        #         print('Sufficiency ', sufficiency)
        #         print('f1 between prediction of proposed model and SVM w/ both inp and exp {}'.format(f1_score(y_test, y_pred, average='macro')))
        #         print('f1 between prediction of proposed model and SVM w/ only inp {}'.format(f1_score(y_test, y_pred_, average='macro')))
        #         print('f1 between prediction of proposed model and SVM w/ only exp {}'.format(f1_score(y_test, y_pred__, average='macro')))
        vals.append(.5*(a/l)+.5*(b/nl))
        k1.append(f1_score(y_test, y_pred, average='macro'))
        k2.append(f1_score(y_test, y_pred_, average='macro'))
        k3.append(f1_score(y_test, y_pred__, average='macro'))
        
        
    print('Intra mean ', np.mean(ID), 'Intra std ', np.std(ID))
    print('Inter mean ', np.mean(ID1), 'Inter std ', np.std(ID1))
    #     print('LAS mean ', np.mean(vals), 'LAS std ', np.std(vals))
    #     print('LUS mean ', np.mean(vals_lus), 'LUS std ', np.std(vals_lus))
    #     print('Comprehensiveness mean ', np.mean(C), 'Comprehensiveness std ', np.std(C))
    #     print('Sufficiency mean ', np.mean(S), 'Sufficiency std ', np.std(S))
    #     print('Avg. f1 between prediction of proposed model and SVM w/ both inp and exp {}'.format(np.mean(k1)))
    #     print('Avg. f1 between prediction of proposed model and SVM w/ only inp {}'.format(np.mean(k2)))
    #     print('Avg. f1 between prediction of proposed model and SVM w/ only exp {}'.format(np.mean(k3)))
    
            
        
        
        
        
        
        
        
        
        
        
        
        
        
    #     y_pred = cross_val_predict(clf, X, y, cv=3)
    #     y_pred_ = cross_val_predict(clf, X_, y, cv=3)
    #     y_pred__ = cross_val_predict(clf, X__, y, cv=3)

    #     print(y_pred)
    #     print(y_pred.shape)

    #     print(y_pred_)
    #     print(y_pred_.shape)

    #     print(0/0)
    
    
    

        

In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import *
import pickle
set_seed(42)
rnd = np.random.randn(25)

def get_repr(i,CNT):
    tokens = i.split('\t')[:-1] if i!='' else ['']
    #print(tokens)

    word_embeddings = []

    for token in tokens:
        if token in glove_vectors.index_to_key:
            word_embeddings.append(glove_vectors[token])
    
    # Calculate the average embedding
    if word_embeddings:
        avg_embedding = np.mean(word_embeddings, axis=0)
    else:
        avg_embedding = rnd
        CNT+=1
        
    return avg_embedding, CNT
    
for i in os.listdir('/kaggle/input/full-ops'):
    print(i)
    CNT=0
    with open('/kaggle/input/full-ops/'+i, 'rb') as f:
        lod = pickle.load(f)
        y = []
        k = []
        p = []
        for i in test_dataset:
            y.append(i[2])
            embed, CNT = get_repr(lod[0][i[3]], CNT)
            k.append(embed)
            a,b= i[0],i[1]
            p.append(np.concatenate([a,b],axis=-1))
    
    print(CNT)
    k = np.asarray(k)
    p = np.asarray(p)
    
    X = np.concatenate([k,p],axis=-1) # inp w/ clip representation w/ exp -> w/ both
    X_ = p # inp w/ clip representation w/o exp -> w/ only input
    X__ = k # inp w/o clip representation w/ exp -> w/ only exp
    
    from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
    from sklearn import svm
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    #clf = svm.SVC(kernel='poly', C=2, random_state=42)
    
    kf = KFold(n_splits=4)
    y = np.array(y)
    vals = []
    vals_lus = []
    k1,k2,k3 = [],[],[]
    C,S = [],[]
    for train, test in kf.split(X):
        
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
        X_train_, X_test_, y_train, y_test = X_[train], X_[test], y[train], y[test]
        X_train__, X_test__, y_train, y_test = X__[train], X__[test], y[train], y[test]
        
        
        clf1 = make_pipeline(StandardScaler(),svm.SVC(probability=True, kernel='rbf', C=2, random_state=42))
        clf1.fit(X_train, y_train)
        y_pred = clf1.predict(X_test)
        
        clf2 = make_pipeline(StandardScaler(),svm.SVC(probability=True,kernel='rbf', C=2, random_state=42))
        clf2.fit(X_train_, y_train)
        y_pred_ = clf2.predict(X_test_)
        
        clf3 = make_pipeline(StandardScaler(), svm.SVC(probability=True,kernel='rbf', C=2, random_state=42))
        clf3.fit(X_train__, y_train)
        y_pred__ = clf3.predict(X_test__)
        
        
        y_pred_proba_w_both = clf1.predict_proba(X_test)
        y_pred_proba_w_inp = clf2.predict_proba(X_test_)
        y_pred_proba_w_exp = clf3.predict_proba(X_test__)
        
        
        
        comprehensiveness, sufficiency = 0,0
        
        counter = 0
        for pred_proba_w_both, pred_proba_w_inp, pred_proba_w_exp, pred_class in zip(y_pred_proba_w_both,y_pred_proba_w_inp,y_pred_proba_w_exp,y_pred):
            comprehensiveness += (pred_proba_w_both[pred_class] - pred_proba_w_inp[pred_class])
            sufficiency += (pred_proba_w_both[pred_class] - pred_proba_w_exp[pred_class])
            counter+=1
        
        
        comprehensiveness = comprehensiveness / counter
        sufficiency = sufficiency / counter
            
        C.append(comprehensiveness)
        S.append(sufficiency)
            
            
            
       
        
        l, nl = 0,0
        us = 0
        for i ,j in zip(y_pred__,y_test):
            if i==j:
                l+=1
            else:
                nl+=1
        a,b=0,0
        for i,j,k,x in zip(y_pred,y_pred_,y_test,y_pred__):
            us+=int(i==k) - int(j==k)
            if k==x:
                a += int(i==k) - int(j==k)
            elif k!=x:
                b += int(i==k) - int(j==k)
        
        print('LAS ',.5*(a/l)+.5*(b/nl))
        print('Leakage Unadjusted Simulatability (LUS)', (us/(l+nl)))
        vals_lus.append(us/(l+nl))
        print('Comprehensiveness ', comprehensiveness)
        print('Sufficiency ', sufficiency)
        print('f1 between prediction of proposed model and SVM w/ both inp and exp {}'.format(f1_score(y_test, y_pred, average='macro')))
        print('f1 between prediction of proposed model and SVM w/ only inp {}'.format(f1_score(y_test, y_pred_, average='macro')))
        print('f1 between prediction of proposed model and SVM w/ only exp {}'.format(f1_score(y_test, y_pred__, average='macro')))
        vals.append(.5*(a/l)+.5*(b/nl))
        k1.append(f1_score(y_test, y_pred, average='macro'))
        k2.append(f1_score(y_test, y_pred_, average='macro'))
        k3.append(f1_score(y_test, y_pred__, average='macro'))
    print('LAS mean ', np.mean(vals), 'LAS std ', np.std(vals))
    print('LUS mean ', np.mean(vals_lus), 'LUS std ', np.std(vals_lus))
    print('Comprehensiveness mean ', np.mean(C), 'Comprehensiveness std ', np.std(C))
    print('Sufficiency mean ', np.mean(S), 'Sufficiency std ', np.std(S))
    print('Avg. f1 between prediction of proposed model and SVM w/ both inp and exp {}'.format(np.mean(k1)))
    print('Avg. f1 between prediction of proposed model and SVM w/ only inp {}'.format(np.mean(k2)))
    print('Avg. f1 between prediction of proposed model and SVM w/ only exp {}'.format(np.mean(k3)))
    
            
        
        
        
        
        
        
        
        
        
        
        
        
        
    #     y_pred = cross_val_predict(clf, X, y, cv=3)
    #     y_pred_ = cross_val_predict(clf, X_, y, cv=3)
    #     y_pred__ = cross_val_predict(clf, X__, y, cv=3)

    #     print(y_pred)
    #     print(y_pred.shape)

    #     print(y_pred_)
    #     print(y_pred_.shape)

    #     print(0/0)
    
    
    

        

In [ ]:
import matplotlib.pyplot as plt

# Mean values for metric A for three models
mean_values = [0.023, 0.019, 0.019, 0.027]

# Three different setups (you should have similar arrays for these setups)
setup1 = [0.012, 0.016, 0.023, 0.02]  # Replace with your actual values
setup2 = [0.018, -0.005, 0.019, 0.022]  # Replace with your actual values


# X-axis labels (if needed)
models = ['500', '1500', '2500', '3500']

# Plotting the graph
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed

# Plot each setup as a separate line
plt.plot(models, mean_values, label='epsilon=0.1')
plt.plot(models, setup1, label='epsilon=None')
plt.plot(models, setup2, label='epsilon=0.01')

# Adding labels and title
plt.xlabel('Models')
plt.ylabel('Mean Value of Metric A')
plt.title('Mean Value of Metric A for Different Setups')

# Adding a legend
plt.legend()

# Display the plot
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Mean values for metric A for three models
mean_values = [0.023, 0.019, 0.019, 0.027]

# Three different setups (you should have similar arrays for these setups)
setup1 = [0.012, 0.016, 0.023, 0.02]  # Replace with your actual values
setup2 = [0.018, -0.005, 0.019, 0.022]  # Replace with your actual values


# X-axis labels (if needed)
models = ['500', '1500', '2500', '3500']

# Plotting the graph
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed

# Plot each setup as a separate line
plt.plot(models, mean_values, label='epsilon=0.1')
plt.plot(models, setup1, label='epsilon=None')
plt.plot(models, setup2, label='epsilon=0.01')

# Adding labels and title
plt.xlabel('Models')
plt.ylabel('Mean Value of Metric A')
plt.title('Mean Value of Metric A for Different Setups')

# Adding a legend
plt.legend()

# Display the plot
plt.grid(True)
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Mean values for metric A for three models
mean_values = [0.1, 0.2, 0.3]

# Three different setups (you should have similar arrays for these setups)
setup1 = [0.15, 0.25, 0.35]  # Replace with your actual values
setup2 = [0.12, 0.22, 0.32]  # Replace with your actual values
setup3 = [0.18, 0.28, 0.38]  # Replace with your actual values

# Create a DataFrame for the data
import pandas as pd

data = pd.DataFrame({
    'Models': ['Model 1', 'Model 2', 'Model 3'],
    'Setup 1': mean_values,
    'Setup 2': setup1,
    'Setup 3': setup2
})

# Set the style for the plot (optional)
sns.set(style='whitegrid')

# Create the line plot using Seaborn
plt.figure(figsize=(10, 6))
sns.lineplot(x='Models', y='value', hue='variable', data=pd.melt(data, ['Models']))

# Adding labels and title
plt.xlabel('Models')
plt.ylabel('Mean Value of Metric A')
plt.title('Mean Value of Metric A for Different Setups')

# Display the plot
plt.legend(title='Setups')
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import *
set_seed(42)
rnd = np.random.randn(25)

def get_repr(i,CNT):
    tokens = i.split('\t')[:-1] if i!='' else ['']
    #print(tokens)

    word_embeddings = []

    for token in tokens:
        if token in glove_vectors.index_to_key:
            word_embeddings.append(glove_vectors[token])
    
    # Calculate the average embedding
    if word_embeddings:
        avg_embedding = np.mean(word_embeddings, axis=0)
    else:
        avg_embedding = rnd
        CNT+=1
        
    return avg_embedding, CNT
    
for i in os.listdir('/kaggle/input/extra-full-ops'):
    print(i)
    CNT=0
    with open('/kaggle/input/extra-full-ops/'+i, 'rb') as f:
        lod = pickle.load(f)
        y = lod[1]
        k = []
        p = []
        for i in test_dataset:
            embed, CNT = get_repr(lod[0][i[3]], CNT)
            k.append(embed)
            a,b= i[0],i[1]
            p.append(np.concatenate([a,b],axis=-1))
    
    print(CNT)
    k = np.asarray(k)
    p = np.asarray(p)
    
    X = np.concatenate([k,p],axis=-1) # inp w/ clip representation w/ exp -> w/ both
    X_ = p # inp w/ clip representation w/o exp -> w/ only input
    X__ = k # inp w/o clip representation w/ exp -> w/ only exp
    
    from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
    from sklearn import svm
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    #clf = svm.SVC(kernel='poly', C=2, random_state=42)
    
    kf = KFold(n_splits=4)
    y = np.array(y)
    vals = []
    k1,k2,k3 = [],[],[]
    for train, test in kf.split(X):
        
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
        X_train_, X_test_, y_train, y_test = X_[train], X_[test], y[train], y[test]
        X_train__, X_test__, y_train, y_test = X__[train], X__[test], y[train], y[test]
        
        
        clf1 = make_pipeline(StandardScaler(),svm.SVC(kernel='rbf', C=2, random_state=42))
        clf1.fit(X_train, y_train)
        y_pred = clf1.predict(X_test)
        
        clf2 = make_pipeline(StandardScaler(),svm.SVC(kernel='rbf', C=2, random_state=42))
        clf2.fit(X_train_, y_train)
        y_pred_ = clf2.predict(X_test_)
        
        clf3 = make_pipeline(StandardScaler(), svm.SVC(kernel='rbf', C=2, random_state=42))
        clf3.fit(X_train__, y_train)
        y_pred__ = clf3.predict(X_test__)
        
        l, nl = 0,0
        
        for i ,j in zip(y_pred__,y_test):
            if i==j:
                l+=1
            else:
                nl+=1
        a,b=0,0
        for i,j,k,x in zip(y_pred,y_pred_,y_test,y_pred__):
            if k==x:
                a += int(i==k) - int(j==k)
            elif k!=x:
                b += int(i==k) - int(j==k)
        
        print('LAS, ',.5*(a/l)+.5*(b/nl))
        print('f1 between prediction of proposed model and SVM w/ both inp and exp {}'.format(f1_score(y_test, y_pred, average='macro')))
        print('f1 between prediction of proposed model and SVM w/ only inp {}'.format(f1_score(y_test, y_pred_, average='macro')))
        print('f1 between prediction of proposed model and SVM w/ only exp {}'.format(f1_score(y_test, y_pred__, average='macro')))
        vals.append(.5*(a/l)+.5*(b/nl))
        k1.append(f1_score(y_test, y_pred, average='macro'))
        k2.append(f1_score(y_test, y_pred_, average='macro'))
        k3.append(f1_score(y_test, y_pred__, average='macro'))
    print('LAS mean ', np.mean(vals), 'LAS std ', np.std(vals))
    print('Avg. f1 between prediction of proposed model and SVM w/ both inp and exp {}'.format(np.mean(k1)))
    print('Avg. f1 between prediction of proposed model and SVM w/ only inp {}'.format(np.mean(k2)))
    print('Avg. f1 between prediction of proposed model and SVM w/ only exp {}'.format(np.mean(k3)))
    
            
        
        
        
        
        
        
        
        
        
        
        
        
        
    #     y_pred = cross_val_predict(clf, X, y, cv=3)
    #     y_pred_ = cross_val_predict(clf, X_, y, cv=3)
    #     y_pred__ = cross_val_predict(clf, X__, y, cv=3)

    #     print(y_pred)
    #     print(y_pred.shape)

    #     print(y_pred_)
    #     print(y_pred_.shape)

    #     print(0/0)
    
    
    

        

In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import *
set_seed(42)
rnd = np.random.randn(25)

def get_repr(i,CNT):
    tokens = i.split('\t')[:-1] if i!='' else ['']
    #print(tokens)

    word_embeddings = []

    for token in tokens:
        if token in glove_vectors.index_to_key:
            word_embeddings.append(glove_vectors[token])
    
    # Calculate the average embedding
    if word_embeddings:
        avg_embedding = np.mean(word_embeddings, axis=0)
    else:
        avg_embedding = rnd
        CNT+=1
        
    return avg_embedding, CNT
    
for i in os.listdir('/kaggle/input/extra-ur-ur'):
    print(i)
    CNT=0
    with open('/kaggle/input/extra-ur-ur/'+i, 'rb') as f:
        lod = pickle.load(f)
        y = lod[1]
        k = []
        p = []
        for i in test_dataset:
            embed, CNT = get_repr(lod[0][i[3]], CNT)
            k.append(embed)
            a,b= i[0],i[1]
            p.append(np.concatenate([a,b],axis=-1))
    
    print(CNT)
    k = np.asarray(k)
    p = np.asarray(p)
    
    X = np.concatenate([k,p],axis=-1) # inp w/ clip representation w/ exp -> w/ both
    X_ = p # inp w/ clip representation w/o exp -> w/ only input
    X__ = k # inp w/o clip representation w/ exp -> w/ only exp
    
    from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
    from sklearn import svm
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    #clf = svm.SVC(kernel='poly', C=2, random_state=42)
    
    kf = KFold(n_splits=4)
    y = np.array(y)
    vals = []
    k1,k2,k3 = [],[],[]
    for train, test in kf.split(X):
        
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
        X_train_, X_test_, y_train, y_test = X_[train], X_[test], y[train], y[test]
        X_train__, X_test__, y_train, y_test = X__[train], X__[test], y[train], y[test]
        
        
        clf1 = make_pipeline(StandardScaler(),svm.SVC(kernel='rbf', C=2, random_state=42))
        clf1.fit(X_train, y_train)
        y_pred = clf1.predict(X_test)
        
        clf2 = make_pipeline(StandardScaler(),svm.SVC(kernel='rbf', C=2, random_state=42))
        clf2.fit(X_train_, y_train)
        y_pred_ = clf2.predict(X_test_)
        
        clf3 = make_pipeline(StandardScaler(), svm.SVC(kernel='rbf', C=2, random_state=42))
        clf3.fit(X_train__, y_train)
        y_pred__ = clf3.predict(X_test__)
        
        l, nl = 0,0
        
        for i ,j in zip(y_pred__,y_test):
            if i==j:
                l+=1
            else:
                nl+=1
        a,b=0,0
        for i,j,k,x in zip(y_pred,y_pred_,y_test,y_pred__):
            if k==x:
                a += int(i==k) - int(j==k)
            elif k!=x:
                b += int(i==k) - int(j==k)
        
        print('LAS, ',.5*(a/l)+.5*(b/nl))
        print('f1 between prediction of proposed model and SVM w/ both inp and exp {}'.format(f1_score(y_test, y_pred, average='macro')))
        print('f1 between prediction of proposed model and SVM w/ only inp {}'.format(f1_score(y_test, y_pred_, average='macro')))
        print('f1 between prediction of proposed model and SVM w/ only exp {}'.format(f1_score(y_test, y_pred__, average='macro')))
        vals.append(.5*(a/l)+.5*(b/nl))
        k1.append(f1_score(y_test, y_pred, average='macro'))
        k2.append(f1_score(y_test, y_pred_, average='macro'))
        k3.append(f1_score(y_test, y_pred__, average='macro'))
    print('LAS mean ', np.mean(vals), 'LAS std ', np.std(vals))
    print('Avg. f1 between prediction of proposed model and SVM w/ both inp and exp {}'.format(np.mean(k1)))
    print('Avg. f1 between prediction of proposed model and SVM w/ only inp {}'.format(np.mean(k2)))
    print('Avg. f1 between prediction of proposed model and SVM w/ only exp {}'.format(np.mean(k3)))
    
            
        
        
        
        
        
        
        
        
        
        
        
        
        
    #     y_pred = cross_val_predict(clf, X, y, cv=3)
    #     y_pred_ = cross_val_predict(clf, X_, y, cv=3)
    #     y_pred__ = cross_val_predict(clf, X__, y, cv=3)

    #     print(y_pred)
    #     print(y_pred.shape)

    #     print(y_pred_)
    #     print(y_pred_.shape)

    #     print(0/0)
    
    
    

        

In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import *
set_seed(42)
rnd = np.random.randn(25)

def get_repr(i,CNT):
    tokens = i.split('\t')[:-1] if i!='' else ['']
    #print(tokens)

    word_embeddings = []

    for token in tokens:
        if token in glove_vectors.index_to_key:
            word_embeddings.append(glove_vectors[token])
    
    # Calculate the average embedding
    if word_embeddings:
        avg_embedding = np.mean(word_embeddings, axis=0)
    else:
        avg_embedding = rnd
        CNT+=1
        
    return avg_embedding, CNT
    
for i in os.listdir('/kaggle/input/final-extra'):
    print(i)
    CNT=0
    with open('/kaggle/input/final-extra/'+i, 'rb') as f:
        lod = pickle.load(f)
        y = lod[1]
        k = []
        p = []
        for i in test_dataset:
            embed, CNT = get_repr(lod[0][i[3]], CNT)
            k.append(embed)
            a,b= i[0],i[1]
            p.append(np.concatenate([a,b],axis=-1))
    
    print(CNT)
    k = np.asarray(k)
    p = np.asarray(p)
    
    X = np.concatenate([k,p],axis=-1) # inp w/ clip representation w/ exp -> w/ both
    X_ = p # inp w/ clip representation w/o exp -> w/ only input
    X__ = k # inp w/o clip representation w/ exp -> w/ only exp
    
    from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
    from sklearn import svm
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    #clf = svm.SVC(kernel='poly', C=2, random_state=42)
    
    kf = KFold(n_splits=4)
    y = np.array(y)
    vals = []
    k1,k2,k3 = [],[],[]
    for train, test in kf.split(X):
        
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
        X_train_, X_test_, y_train, y_test = X_[train], X_[test], y[train], y[test]
        X_train__, X_test__, y_train, y_test = X__[train], X__[test], y[train], y[test]
        
        
        clf1 = make_pipeline(StandardScaler(),svm.SVC(kernel='rbf', C=2, random_state=42))
        clf1.fit(X_train, y_train)
        y_pred = clf1.predict(X_test)
        
        clf2 = make_pipeline(StandardScaler(),svm.SVC(kernel='rbf', C=2, random_state=42))
        clf2.fit(X_train_, y_train)
        y_pred_ = clf2.predict(X_test_)
        
        clf3 = make_pipeline(StandardScaler(), svm.SVC(kernel='rbf', C=2, random_state=42))
        clf3.fit(X_train__, y_train)
        y_pred__ = clf3.predict(X_test__)
        
        l, nl = 0,0
        
        for i ,j in zip(y_pred__,y_test):
            if i==j:
                l+=1
            else:
                nl+=1
        a,b=0,0
        for i,j,k,x in zip(y_pred,y_pred_,y_test,y_pred__):
            if k==x:
                a += int(i==k) - int(j==k)
            elif k!=x:
                b += int(i==k) - int(j==k)
        
        print('LAS, ',.5*(a/l)+.5*(b/nl))
        print('f1 between prediction of proposed model and SVM w/ both inp and exp {}'.format(f1_score(y_test, y_pred, average='macro')))
        print('f1 between prediction of proposed model and SVM w/ only inp {}'.format(f1_score(y_test, y_pred_, average='macro')))
        print('f1 between prediction of proposed model and SVM w/ only exp {}'.format(f1_score(y_test, y_pred__, average='macro')))
        vals.append(.5*(a/l)+.5*(b/nl))
        k1.append(f1_score(y_test, y_pred, average='macro'))
        k2.append(f1_score(y_test, y_pred_, average='macro'))
        k3.append(f1_score(y_test, y_pred__, average='macro'))
    print('LAS mean ', np.mean(vals), 'LAS std ', np.std(vals))
    print('Avg. f1 between prediction of proposed model and SVM w/ both inp and exp {}'.format(np.mean(k1)))
    print('Avg. f1 between prediction of proposed model and SVM w/ only inp {}'.format(np.mean(k2)))
    print('Avg. f1 between prediction of proposed model and SVM w/ only exp {}'.format(np.mean(k3)))
    
            
        
        
        
        
        
        
        
        
        
        
        
        
        
    #     y_pred = cross_val_predict(clf, X, y, cv=3)
    #     y_pred_ = cross_val_predict(clf, X_, y, cv=3)
    #     y_pred__ = cross_val_predict(clf, X__, y, cv=3)

    #     print(y_pred)
    #     print(y_pred.shape)

    #     print(y_pred_)
    #     print(y_pred_.shape)

    #     print(0/0)
    
    
    

        

In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
set_seed(42)
rnd = np.random.randn(25)
def get_repr(i):
    tokens = i.split('\t')[:-1] if i!='' else ['']
    #print(tokens)

    word_embeddings = []

    for token in tokens:
        if token in glove_vectors.index_to_key:
            word_embeddings.append(glove_vectors[token])
    
    # Calculate the average embedding
    if word_embeddings:
        avg_embedding = np.mean(word_embeddings, axis=0)
    else:
        avg_embedding = rnd
        
    return avg_embedding
    
for i in os.listdir('/kaggle/input/full-ops'):
    print(i)
    with open('/kaggle/input/full-ops/'+i, 'rb') as f:
        lod = pickle.load(f)
        y = lod[1]
        k = []
        p = []
        for i in test_dataset:
            k.append(get_repr(lod[0][i[3]]))
            a,b= i[0],i[1]
            p.append(np.concatenate([a,b],axis=-1))

    k = np.asarray(k)
    p = np.asarray(p)
    
    X = np.concatenate([k,p],axis=-1) # inp w/ clip representation w/ exp
    X_ = p # inp w/ clip representation w/o exp
    X__ = k # inp w/o clip representation w/ exp
    
    from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
    from sklearn import svm
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    #clf = svm.SVC(kernel='poly', C=2, random_state=42)
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
    kf = KFold(n_splits=2)
    y = np.array(y)
    vals = []
    for train, test in kf.split(X):
        
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
        X_train_, X_test_, y_train, y_test = X_[train], X_[test], y[train], y[test]
        X_train__, X_test__, y_train, y_test = X__[train], X__[test], y[train], y[test]
        
        
        clf1 = make_pipeline(StandardScaler(),svm.SVC(kernel='rbf', C=4, random_state=42))
        clf1.fit(X_train, y_train)
        y_pred = clf1.predict(X_test)
        
        clf2 = make_pipeline(StandardScaler(),svm.SVC(kernel='rbf', C=4, random_state=42))
        clf2.fit(X_train_, y_train)
        y_pred_ = clf2.predict(X_test_)
        
        clf3 = make_pipeline(StandardScaler(), svm.SVC(kernel='rbf', C=4, random_state=42))
        clf3.fit(X_train__, y_train)
        y_pred__ = clf3.predict(X_test__)
        
        l, nl = 0,0
        
        for i ,j in zip(y_pred__,y_test):
            if i==j:
                l+=1
            else:
                nl+=1
        a,b=0,0
        for i,j,k,x in zip(y_pred,y_pred_,y_test,y_pred__):
            if k==x:
                a += int(i==k) - int(j==k)
            elif k!=x:
                b += int(i==k) - int(j==k)
        
        print(.5*(a/l)+.5*(b/nl))
        vals.append(.5*(a/l)+.5*(b/nl))
    print('mean ', np.mean(vals), 'std ', np.std(vals))
    
            
        
        
        
        
        
        
        
        
        
        
        
        
    #     y_pred = cross_val_predict(clf, X, y, cv=3)
    #     y_pred_ = cross_val_predict(clf, X_, y, cv=3)
    #     y_pred__ = cross_val_predict(clf, X__, y, cv=3)

    #     print(y_pred)
    #     print(y_pred.shape)

    #     print(y_pred_)
    #     print(y_pred_.shape)

    #     print(0/0)
    
    
    

        

In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
def get_repr(i):
    tokens = i.split('\t')[:-1] if i!='' else ['']
    #print(tokens)

    word_embeddings = []

    for token in tokens:
        if token in glove_vectors.index_to_key:
            word_embeddings.append(glove_vectors[token])

    # Calculate the average embedding
    if word_embeddings:
        avg_embedding = np.mean(word_embeddings, axis=0)
    else:
        avg_embedding = rnd
        
    return avg_embedding
    
for i in os.listdir('/kaggle/input/extras'):
    print(i)
    with open('/kaggle/input/extras/'+i, 'rb') as f:
        lod = pickle.load(f)
        y = lod[1]
        k = []
        p = []
        for i in test_dataset_1:
            k.append(get_repr(lod[0][i[3]]))
            a,b= i[0],i[1]
            p.append(np.concatenate([a,b],axis=-1))

    k = np.asarray(k)
    p = np.asarray(p)
    
    X = np.concatenate([k,p],axis=-1) # inp w/ clip representation w/ exp
    X_ = p # inp w/ clip representation w/o exp
    X__ = k # inp w/o clip representation w/ exp
    
    from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
    from sklearn import svm
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    #clf = svm.SVC(kernel='poly', C=2, random_state=42)
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
    kf = KFold(n_splits=5)
    y = np.array(y)
    vals = []
    for train, test in kf.split(X):
        
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
        X_train_, X_test_, y_train, y_test = X_[train], X_[test], y[train], y[test]
        X_train__, X_test__, y_train, y_test = X__[train], X__[test], y[train], y[test]
        
        
        clf1 = make_pipeline(StandardScaler(),GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0))
        clf1.fit(X_train, y_train)
        y_pred = clf1.predict(X_test)
        
        clf2 = make_pipeline(StandardScaler(),GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0))
        clf2.fit(X_train_, y_train)
        y_pred_ = clf2.predict(X_test_)
        
        clf3 = make_pipeline(StandardScaler(),GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0))
        clf3.fit(X_train__, y_train)
        y_pred__ = clf3.predict(X_test__)
        
        l, nl = 0,0
        
        for i ,j in zip(y_pred__,y_test):
            if i==j:
                l+=1
            else:
                nl+=1
        a,b=0,0
        for i,j,k,x in zip(y_pred,y_pred_,y_test,y_pred__):
            if k==x:
                a += int(i==k) - int(j==k)
            elif k!=x:
                b += int(i==k) - int(j==k)
        
        vals.append(.5*(a/l)+.5*(b/nl))
        print(.5*(a/l)+.5*(b/nl))
    print('mean ', np.mean(vals), 'std ', np.std(vals))
    
            
        
        
        
        
        
        
        
        
        
        
        
        
    #     y_pred = cross_val_predict(clf, X, y, cv=3)
    #     y_pred_ = cross_val_predict(clf, X_, y, cv=3)
    #     y_pred__ = cross_val_predict(clf, X__, y, cv=3)

    #     print(y_pred)
    #     print(y_pred.shape)

    #     print(y_pred_)
    #     print(y_pred_.shape)

    #     print(0/0)
    
    
    

        

In [ ]:
a = np.array([[1,2],[0,8]])
a[np.array([0])]

In [ ]:
p.shape

In [ ]:
if p:
    print(1)

In [ ]:
0.33062397372742197 0.3229131047815565

In [ ]:
1193514

In [ ]:
leakage = []
non_leakage = []
global_dict = {}
for ii,ti, gol, ids in tqdm(test_dataset_1):
    we = []
    ii = ii.unsqueeze(dim=0)
    ti = ti.unsqueeze(dim=0)
    gol = [gol]
    ids = [ids]
    
    for id_ in ids: 
        tokens = [i for i in d_ids[id_].split('\t') if i!='']
        #print(tokens)

        word_embeddings = []

        for token in tokens:
            if token in glove_vectors.index_to_key:
                word_embeddings.append(glove_vectors[token])

        # Calculate the average embedding
        if word_embeddings:
            avg_embedding = np.mean(word_embeddings, axis=0)
        else:
            # Handle the case where none of the words were found in the vocabulary
            avg_embedding = np.zeros(25)  # You can use a zero vector or other defaults

        we.append(avg_embedding)

    we = np.asarray(we)


    #print(we.shape)





    i = ii.float().to(device)
    t = ti.float().to(device)


    batch_size = i.shape[0]
    optimizer.zero_grad()

    logits, m = tinymodel(i,t, torch.tensor(we).to(device), train=False)
    logits_no_exp,_ = tinymodel(i,t, torch.tensor(we).to(device), train=False, no_exp=True)
    logits_only_exp,_ = tinymodel(i,t, torch.tensor(we).to(device), train=False,only_exp=True)
    
    
  
    #print(gol)
    #print(logits_only_exp.argmax(dim=-1).item(), logits.argmax(dim=-1).item(), logits_no_exp.argmax(dim=-1).item())
    
    if gol[0]==logits_only_exp.argmax(dim=-1).item():
        leakage.append(ids[0])
    elif gol[0]!=logits_only_exp.argmax(dim=-1).item():
        non_leakage.append(ids[0]) 
    
    l = int(gol[0]==logits.argmax(dim=-1).item()) - int(gol[0]==logits_no_exp.argmax(dim=-1).item())
    
    global_dict[ids[0]] = l
        
    
    #print(0/0)
    

In [ ]:
l1,l2 = 0,0
for i in leakage:
    l1+=global_dict[i]
l1 = l1/len(leakage)

for i in non_leakage:
    l2+=global_dict[i]
l2 = l2/len(non_leakage)

In [ ]:
0.5*l1+0.5*l2

In [ ]:
l1,l2

In [ ]:
len(leakage)

In [ ]:
set_seed(42)
al, alc,ti,op,tt,t_id = get_performance_test()

In [ ]:
torch.tensor(tokenizer1.encode('[KB] trump are'))

In [ ]:
al, alc,ti,op,tt,t_id = get_performance_test()

In [ ]:
s = ['when', 'why', 'our']
final_tokens = [
   "cops",
"police",
"gun",
"logical",
"criminal",
"pos",
"law",
"logic",
"crime",
"unarmed",
"comedy",
"common",
"criminals",
"you","when", "us"
]

final_tokens = [i for i in final_tokens if (i not in s and len(i)>2)]

In [ ]:
final_tokens

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# from gensim import corpora, models

# def extract_central_concept_lda(token_list, num_topics=2, passes=10):
#     # Create a dictionary from the token list
#     dictionary = corpora.Dictionary([token_list])
    
#     # Create a document-term matrix
#     doc_term_matrix = [dictionary.doc2bow(token_list)]
    
#     # Create an LDA model
#     lda_model = models.LdaModel(doc_term_matrix, num_topics=num_topics, id2word=dictionary, passes=passes)
    
#     # Extract the top words from all topics
#     topic_words = []
#     for topic_id in range(num_topics):
#         topic_words.extend([word for word, _ in lda_model.show_topic(topic_id)])
    
#     # Calculate the most common word among all topics
#     central_concept = max(set(topic_words), key=topic_words.count)
    
#     return central_concept

# # Example token list
# token_list = ["apple", "banana", "apple", "orange", "banana", "apple", "grape"]

# central_concept_lda = extract_central_concept_lda(token_list)
# print("Central Concept (LDA):", central_concept_lda)


In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
import spacy

# Load pre-trained word vectors model
nlp = spacy.load("en_core_web_md")




from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from collections import Counter

def extract_central_concept_word2vec(token_list):
    # Train a Word2Vec model
    model = Word2Vec([token_list], vector_size=100, window=5, min_count=1, sg=1)
    
    # Calculate the average vector for all tokens
    #token_vectors = [model.wv[token] for token in token_list]
    #avg_vector = np.mean(token_vectors, axis=0)
    token_vectors = [nlp(token).vector for token in token_list]
    avg_vector = sum(token_vectors) / len(token_vectors)
    similar_words = nlp.vocab.vectors.most_similar(avg_vector.reshape(1, -1), n=10)
    print(similar_words)
    for i in similar_words[0][0]:
        print(nlp.vocab.strings[i])
    # Calculate cosine similarities between average vector and token vectors
    #     similarities = cosine_similarity([avg_vector], token_vectors)

    #     # Create a frequency counter for the words
    #     word_frequencies = Counter(token_list)

    #     # Find the most frequent word among the most similar tokens
    #     central_word_index = np.argmax(similarities)
    #     central_word = token_list[central_word_index]

    #     return central_word




# Example token list
token_list = ['qaida',
'jihadist',
'iraqi',
'islamist',
'islamic',
'muslim',
'islam',
'jihad']

central_concept_word_vectors = extract_central_concept_word2vec(token_list)
print("Central Concept (Word Vectors):", central_concept_word_vectors)


In [ ]:
nlp.vocab.vectors.most_similar

In [ ]:
!git clone https://github.com/FreddeFrallan/Non-Residual-Prompting

In [ ]:
!pip install -r /kaggle/working/Non-Residual-Prompting/requirements.txt

In [ ]:
tt

In [ ]:
!bash '/kaggle/working/Non-Residual-Prompting/inference.sh'

In [ ]:
!python /kaggle/working/Non-Residual-Prompting/InferenceExample.py

In [ ]:
t_id

In [ ]:
output_aware_token_idx = []
output_aware_tokens = []
for i in op.topk(5000).indices:
    k = model1.transformer.wte(i.cuda()).detach().cpu().unsqueeze(1)
    
    #output_aware_token_idx.append(ti.mm(k))
    val = torch.nn.functional.cosine_similarity(ti,k.T)[0].item()
    if val>0:
        output_aware_token_idx.append(val)
        output_aware_tokens.append(tokenizer1.decode(i))
    #     print('{} -> {}'.format(tokenizer1.decode(i), ti.mm(k).squeeze()))
    

In [ ]:
output_aware_tokens = list(map(lambda x: x.strip().lower(), output_aware_tokens))

In [ ]:
output_aware_tokens[0:10]

In [ ]:
output_aware_token_idx[0:10]

In [ ]:
len(output_aware_token_idx)

In [ ]:
# image = preprocess(Image.open('./data/img/28197.png')).unsqueeze(0).to(device)

# clip_feats = []
# for i in tqdm(output_aware_tokens):
#     text = clip.tokenize([i]).to(device)

#     with torch.no_grad():
#         image_features = model.encode_image(image)
#         text_features = model.encode_text(text)
#         clip_feats.append(torch.nn.functional.cosine_similarity(image_features, text_features)[0].item())
#         #clip_feats.append(image_features.mm(text_features.T).squeeze().detach().cpu().item())

    

In [ ]:
image = preprocess(Image.open('./data/'+t_id)).unsqueeze(0).to(device)
text = clip.tokenize(output_aware_tokens).to(device)
# clip_feats = []
text_cand = clip.tokenize(tt).to(device)

    

with torch.no_grad():
    if_ = model.encode_image(image)
    tf_ = model.encode_text(text_cand)
    
    print(if_.shape, tf_.shape)
    multimodal_features = (if_+tf_)/2
    text_features = model.encode_text(text)
    
    clip_feats = torch.nn.functional.cosine_similarity(multimodal_features, text_features, dim=1)


clip_feats = clip_feats.cpu().numpy().tolist()
    

In [ ]:
len(clip_feats)

In [ ]:
output_aware_tokens[0:10]

In [ ]:
clip_feats[0:10]

In [ ]:
max(clip_feats), min(clip_feats)

In [ ]:
output_aware_token_idx[0:10]

In [ ]:
max(output_aware_token_idx), min(output_aware_token_idx)

In [ ]:
combined_feats = []

In [ ]:
for i,j in zip(output_aware_token_idx, clip_feats):
    #tmp = 2*i*j/(i+j)
    tmp = 1.0*j+0.05*i
    #tmp = j
    combined_feats.append(tmp)

In [ ]:
np.argsort(combined_feats)

In [ ]:
for i in np.argsort(combined_feats)[-1:-20:-1]:
    print(output_aware_tokens[i])

In [ ]:
for i in np.argsort(combined_feats)[-1:-10:-1]:
    print(output_aware_tokens[i])

In [ ]:
output_aware_tokens[138]

In [ ]:
image_features.mm(text_features.T)

In [ ]:
image_features.shape

In [ ]:
fp = text_features.mm(image_features.T)

In [ ]:
output_aware_token_idx = torch.stack(output_aware_token_idx).squeeze()

In [ ]:
output_aware_token_idx = output_aware_token_idx.unsqueeze(1) 

In [ ]:
fp.shape

In [ ]:
output_aware_token_idx.shape

In [ ]:
fin_feats = (fp.cpu()+output_aware_token_idx.cpu())/2

In [ ]:
fin_feats.squeeze().argsort()

In [ ]:
for i in fin_feats.squeeze().argsort():
    print(output_aware_tokens[i])

In [ ]:
output_aware_token_idx.argsort()

In [ ]:
all_idx = op.topk(500).indices

In [ ]:
all_idx

In [ ]:
for i in output_aware_token_idx.argsort():
    print(tokenizer1.decode(all_idx[i.item()]))
    

In [ ]:
output_aware_token_idx.argmax()

In [ ]:
tokenizer1.decode(op.topk(50).indices[output_aware_token_idx.argmax().item()])

In [ ]:
tokenizer1.encode('deformity')

In [ ]:
tokenizer1.decode([40720, 48443])

In [ ]:
k = model1.transformer.wte(torch.tensor(tokenizer1.encode('')).cuda()).detach().cpu()

In [ ]:
k = k.mean(dim=0).unsqueeze(0)

In [ ]:
k.shape

In [ ]:
ti.mm(k.T)

In [ ]:
tokens_index = torch.mm(ti, E_.T)[0]

In [ ]:
tokens_index.topk(2)

In [ ]:
itm = tokens_index.topk(2).indices[-1].item()

In [ ]:
itm

In [ ]:
E_ = E.detach().cpu()

In [ ]:
E_ += E.detach().cpu()[itm,:]

In [ ]:
E_.shape

In [ ]:
model1.transformer.wte.weight

In [ ]:
# grad_.shape

In [ ]:
E.T.shape

In [ ]:
E.detach().cpu()

In [ ]:
al, alc,ti,op = get_performance_test()
l2l = {'normal':0, 'offensive':1}



In [ ]:
al, alc = get_performance_test_nm()
l2l = {'normal':0, 'offensive':1}
al_ = list(map(lambda x: l2l[x.strip()], al))
alc_ = list(map(lambda x: l2l[x.strip()], alc))
print(f1_score(gl_test, al_, average='macro'), f1_score(gl_test, alc_, average='macro'))
print(accuracy_score(gl_test, al_), accuracy_score(gl_test, alc_))

In [ ]:
gl_test  = []
for _,_,g,_ in test_dataset:
    gl_test.append(g)

In [ ]:
# gl_test

In [ ]:
# from tqdm import tqdm

In [ ]:
# E[tokenizer1.encode(tokenizer1.eos_token),:]

In [ ]:
# all_labs = []
# for ii in tqdm(range(500)):  
#     dix = {0:'normal', 1:'offensive'}

#     i = im_tensor_[ii,:].unsqueeze(0)
#     t = tx_tensor_[ii,:].unsqueeze(0)
#     #     print(i.shape, t.shape)
#     i = i.float().to(device)
#     t = t.float().to(device)
#     batch_size = i.shape[0]
#     optimizer.zero_grad()
#     optimizer1.zero_grad()
#     logits, m = tinymodel(i,t)
#     # print(m)
#     m = m.unsqueeze(dim=1)

#     gumbel_logits = F.gumbel_softmax(logits, tau=1, hard=True)
#     # print(gumbel_logits)
#     agmx = gumbel_logits.argmax(dim=1)
    

#     # print(agmx, logits.argmax(dim=1))
#     prmpt0, lab0 = get_tokens([tokenizer1.eos_token]*batch_size)
#     prpmt = ['model thinks the meme is ']*batch_size

#     portion1,lab1 = get_tokens(prpmt,begin=True)
#     portion2,lab3 = get_tokens(['output of the classifier multimodal embedding is ']*batch_size)
#     lab4 = [tokenizer1.encode('-')[:] for i in range(batch_size)]
#     lab4 = torch.tensor(lab4)

#     # portion3 = get_tokens(['the meme is actually'])






#     kk = []
#     labs_verbalized = []
#     # batch_prompt = ['the model thinks the meme is ']*32
#     lab2 = []
#     for i in gumbel_logits:
#         agmx = i.argmax()
#         # print(agmx)
#         # print(dix[agmx.item()])
#         tokenized_ = tokenizer1.encode(dix[agmx.item()])[:]
#         labs_verbalized.append(dix[agmx.item()])
#         # print(tokenized_)
#         lab2.append(tokenized_)
#         embedding = E[tokenized_[0]]
#         one_hot = i.view(-1, 1)
#         # print(embedding.shape, one_hot, one_hot.shape)

#         e = torch.sum(one_hot*embedding, dim=0)
#         kk.append(e)
#     lab2 = torch.tensor(lab2)
#     string = ['the meme is actually']
#     portion3,lab5 = get_tokens(string)
#     inp_embed = torch.stack(kk).unsqueeze(dim=1)
#     # print(inp_embed)
#     # print(inp_embed.shape)
#     # print(portion1.shape,inp_embed.shape,portion2.shape,m.shape,portion3.shape)
#     final_embeds = torch.cat((prmpt0,portion1,inp_embed,portion2,m,portion3),dim=1)
#     # print(final_embeds.shape)
#     # print(lab1.shape, lab2.shape, lab3.shape, lab4.shape, lab5.shape)
#     labs = torch.cat((lab0,lab1,lab2,lab3,lab4,lab5),dim=1)
#     #     print(final_embeds.shape, labs.shape)
#     # print(0/0)
#     #     print(tokenizer1.batch_decode(labs))
#     output2 = model1(inputs_embeds = final_embeds.half())
#     #     print(output2.logits.argmax(dim=-1))
#     llm_lab = tokenizer1.decode(output2.logits.argmax(dim=-1)[0][-1])
#     all_labs.append(llm_lab)

In [ ]:
output2.logits.argmax(dim=-1)


In [ ]:
tokenizer1.batch_decode(output2.logits.argmax(dim=-1))

In [ ]:
# all_labs_clf_

In [ ]:
f1_score(gl_test, all_labs_, average='macro')

In [ ]:
accuracy_score(gl_test, all_labs_)

In [ ]:
balanced_accuracy_score(gl_test, all_labs_)

In [ ]:

tokenizer1.decode(tokenizer1.encode('offensive')[1:])

In [ ]:
import torch
torch.manual_seed(0)
import random
random.seed(0)
import numpy as np
np.random.seed(0)
tinymodel = TinyModel(mdl,rand=False).to(device)
# E = model1.transformer.wte.weight.detach()
E = model1.model.model.embed_tokens.weight.detach()
optimizer = optim.Adam(tinymodel.parameters(), lr=0.0005)
optimizer1 = optim.Adam(model1.parameters(), lr=0.0005)
dix = {0:'normal', 1:'offensive'}

# Training loop
num_epochs = 10
def transpose(x):
    return x.transpose(-2, -1)

for epoch in range(num_epochs):
  l=0
  cnt = 0
  l_lm = 0
  l_clf = 0
  tinymodel.train()
  for i,t, gol in dataloader:
      # print(batch_model1, batch_model1)
      i = i.float().to(device)
      t = t.float().to(device)
      batch_size = i.shape[0]
      optimizer.zero_grad()
      optimizer1.zero_grad()
      logits, m = tinymodel(i,t)
      # print(m)
      m = m.unsqueeze(dim=1)

      gumbel_logits = F.gumbel_softmax(logits, tau=1, hard=True)
      # print(gumbel_logits)
      agmx = gumbel_logits.argmax(dim=1)

      # print(agmx, logits.argmax(dim=1))
      prpmt = ['model thinks the meme is ']*batch_size

      portion1,lab1 = get_tokens(prpmt,begin=True)
      portion2,lab3 = get_tokens(['output of the classifier multimodal embedding is ']*batch_size)
      lab4 = [tokenizer1.encode('-')[1:] for i in range(batch_size)]
      lab4 = torch.tensor(lab4)

      # portion3 = get_tokens(['the meme is actually'])






      kk = []
      labs_verbalized = []
      # batch_prompt = ['the model thinks the meme is ']*32
      lab2 = []
      for i in gumbel_logits:
        agmx = i.argmax()
        # print(agmx)
        # print(dix[agmx.item()])
        tokenized_ = tokenizer1.encode(dix[agmx.item()])[1:]
        labs_verbalized.append(dix[agmx.item()])
        # print(tokenized_)
        lab2.append(tokenized_)
        embedding = E[tokenized_[0]]
        one_hot = i.view(-1, 1)
        # print(embedding.shape, one_hot, one_hot.shape)

        e = torch.sum(one_hot*embedding, dim=0)
        kk.append(e)
      lab2 = torch.tensor(lab2)
      string = ['the meme is actually {}'.format(i) for i in labs_verbalized]
      portion3,lab5 = get_tokens(string)
      inp_embed = torch.stack(kk).unsqueeze(dim=1)
      # print(inp_embed)
      # print(inp_embed.shape)
      # print(portion1.shape,inp_embed.shape,portion2.shape,m.shape,portion3.shape)
      final_embeds = torch.cat((portion1,inp_embed,portion2,m,portion3),dim=1)
      # print(final_embeds.shape)
      # print(lab1.shape, lab2.shape, lab3.shape, lab4.shape, lab5.shape)
      labs = torch.cat((lab1,lab2,lab3,lab4,lab5),dim=1)
      print(final_embeds.shape, labs.shape)
      # print(0/0)
      output2 = model1(inputs_embeds = final_embeds.half(), labels = labs.long())
      # print(output2.loss)
      # print(0/0)
      loss_lm = output2.loss




      loss_clf = torch.nn.functional.cross_entropy(logits, gol.to(device), reduction='mean')

      loss = loss_lm+loss_clf

      loss.backward()
      optimizer.step()
      optimizer1.step()


      l+=loss.detach().item()
      l_clf+=loss_clf.detach().item()
      l_lm+=loss_lm.detach().item()
      cnt+=1

  print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {l/cnt}, Loss clf: {l_clf/cnt}, Loss llm: {l_lm/cnt}")
  tinymodel.eval()
  pred,_ = tinymodel(im_tensor_.float().to(device),tx_tensor_.float().to(device))
  pred = pred.argmax(dim=1).detach().cpu().numpy()
  print(f1_score(gl_, pred, average='macro'), accuracy_score(gl_, pred))
print("Training complete!")


In [ ]:
dix = {0:'normal', 1:'offensive'}

i = im_tensor_[0,:].unsqueeze(0)
t = tx_tensor_[0,:].unsqueeze(0)
print(i.shape, t.shape)
i = i.float().to(device)
t = t.float().to(device)
batch_size = i.shape[0]
optimizer.zero_grad()
optimizer1.zero_grad()
logits, m = tinymodel(i,t)
# print(m)
m = m.unsqueeze(dim=1)

gumbel_logits = F.gumbel_softmax(logits, tau=1, hard=True)
# print(gumbel_logits)
agmx = gumbel_logits.argmax(dim=1)

# print(agmx, logits.argmax(dim=1))
prpmt = ['model thinks the meme is ']*batch_size

portion1,lab1 = get_tokens(prpmt,begin=True)
portion2,lab3 = get_tokens(['output of the classifier multimodal embedding is ']*batch_size)
lab4 = [tokenizer1.encode('-')[1:] for i in range(batch_size)]
lab4 = torch.tensor(lab4)

# portion3 = get_tokens(['the meme is actually'])






kk = []
labs_verbalized = []
# batch_prompt = ['the model thinks the meme is ']*32
lab2 = []
for i in gumbel_logits:
  agmx = i.argmax()
  # print(agmx)
  # print(dix[agmx.item()])
  tokenized_ = tokenizer1.encode(dix[agmx.item()])[1:]
  labs_verbalized.append(dix[agmx.item()])
  # print(tokenized_)
  lab2.append(tokenized_)
  embedding = E[tokenized_[0]]
  one_hot = i.view(-1, 1)
  # print(embedding.shape, one_hot, one_hot.shape)

  e = torch.sum(one_hot*embedding, dim=0)
  kk.append(e)
lab2 = torch.tensor(lab2)
string = ['the meme is actually {}'.format(i) for i in labs_verbalized]
portion3,lab5 = get_tokens(string)
inp_embed = torch.stack(kk).unsqueeze(dim=1)
# print(inp_embed)
# print(inp_embed.shape)
# print(portion1.shape,inp_embed.shape,portion2.shape,m.shape,portion3.shape)
final_embeds = torch.cat((portion1,inp_embed,portion2,m,portion3),dim=1)
# print(final_embeds.shape)
# print(lab1.shape, lab2.shape, lab3.shape, lab4.shape, lab5.shape)
labs = torch.cat((lab1,lab2,lab3,lab4,lab5),dim=1)
print(final_embeds.shape, labs.shape)
# print(0/0)
output2 = model1(inputs_embeds = final_embeds.half(), labels = labs.long())

In [ ]:
output2

In [ ]:
# import torch
# torch.manual_seed(0)
# import random
# random.seed(0)
# import numpy as np
# np.random.seed(0)
# tinymodel = TinyModel(mdl_rand,rand=False).to(device)
# optimizer = optim.Adam(tinymodel.parameters(), lr=0.0005)


# # Training loop
# num_epochs = 10
# def transpose(x):
#     return x.transpose(-2, -1)

# for epoch in range(num_epochs):
#   l=0
#   cnt = 0
#   tinymodel.train()
#   for i,t, gol in dataloader:
#       # print(batch_model1, batch_model1)
#       i = i.float().to(device)
#       t = t.float().to(device)

#       optimizer.zero_grad()
#       logits = tinymodel(i,t)
#       # b = model2(batch_model2)



#       loss = torch.nn.functional.cross_entropy(logits, gol.to(device), reduction='mean')
#       loss.backward()
#       optimizer.step()


#       l+=loss.detach().item()
#       cnt+=1

#   print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {l/cnt}")
#   tinymodel.eval()
#   pred = tinymodel(im_tensor_.float().to(device),tx_tensor_.float().to(device))
#   pred = pred.argmax(dim=1).detach().cpu().numpy()
#   print(f1_score(gl_, pred, average='macro'), accuracy_score(gl_, pred))
# print("Training complete!")


In [ ]:
im_tensor_.shape

In [ ]:
tinymodel.eval()
pred = tinymodel(im_tensor_.float().to(device),tx_tensor_.float().to(device))

In [ ]:
pred = pred.argmax(dim=1).detach().cpu().numpy()

In [ ]:
pred

In [ ]:
gl_

In [ ]:
from sklearn.metrics import *
f1_score(gl_, pred, average='macro'), accuracy_score(gl_, pred)

In [ ]:
from sklearn.metrics import *
f1_score(gl_, pred, average='macro'), accuracy_score(gl_, pred)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(multimodal_tensor.numpy(), gold_label, test_size=0.2, random_state=42)

In [ ]:
import numpy as np

In [ ]:
np.shape(X_train), np.shape(X_test)

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(X_train)

pred = kmeans.predict(X_test)


In [ ]:
pred

In [ ]:
# y_test

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(y_test, pred, average='micro')

In [ ]:
im_tensor.shape, tx_tensor.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(torch.cat((im_tensor,tx_tensor),dim=1).numpy(), gold_label, test_size=0.33, random_state=42)

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(X_train)

pred = kmeans.predict(X_test)
f1_score(y_test, pred, average='micro')